Connect to google drive

In [6]:
from google.colab import drive
drive.mount("/content/drive/", force_remount=True)

Mounted at /content/drive/


In [0]:
cd ./drive/My\ Drive/636/data/part4_data/video

/content/drive/My Drive/636/data/part4_data/video


In [0]:
ls

010.mp4  01.mp4   029.mp4  038.mp4  047.mp4  056.mp4  065.mp4  074.mp4  083.mp4
011.mp4  020.mp4  02.mp4   039.mp4  048.mp4  057.mp4  066.mp4  075.mp4  084.mp4
012.mp4  021.mp4  030.mp4  03.mp4   049.mp4  058.mp4  067.mp4  076.mp4  085.mp4
013.mp4  022.mp4  031.mp4  040.mp4  04.mp4   059.mp4  068.mp4  077.mp4  08.mp4
014.mp4  023.mp4  032.mp4  041.mp4  050.mp4  05.mp4   069.mp4  078.mp4  09.mp4
015.mp4  024.mp4  033.mp4  042.mp4  051.mp4  060.mp4  06.mp4   079.mp4
016.mp4  025.mp4  034.mp4  043.mp4  052.mp4  061.mp4  070.mp4  07.mp4
017.mp4  026.mp4  035.mp4  044.mp4  053.mp4  062.mp4  071.mp4  080.mp4
018.mp4  027.mp4  036.mp4  045.mp4  054.mp4  063.mp4  072.mp4  081.mp4
019.mp4  028.mp4  037.mp4  046.mp4  055.mp4  064.mp4  073.mp4  082.mp4


In [0]:
mkdir ../neg_f

In [0]:
%%shell
for i in {1..85}
{
    mkdir ../neg_f/$i
    ffmpeg -i 0$i.mp4 ../neg_f/$i/%d.png
}

ffmpeg version 3.4.6-0ubuntu0.18.04.1 Copyright (c) 2000-2019 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.3.0-16ubuntu3)
  configuration: --prefix=/usr --extra-version=0ubuntu0.18.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --ena

Install Openpose

In [0]:
ls ./drive/My\ Drive/636/data/part8_data_refine_part4/

/content/drive/My Drive/636/data/part8_data_refine_part4


In [0]:
cd /content

/content


In [0]:
import os
from os.path import exists, join, basename, splitext

git_repo_url = 'https://github.com/CMU-Perceptual-Computing-Lab/openpose.git'
project_name = splitext(basename(git_repo_url))[0]
if not exists(project_name):
  # see: https://github.com/CMU-Perceptual-Computing-Lab/openpose/issues/949
  # install new CMake becaue of CUDA10
  !wget -q https://cmake.org/files/v3.13/cmake-3.13.0-Linux-x86_64.tar.gz
  !tar xfz cmake-3.13.0-Linux-x86_64.tar.gz --strip-components=1 -C /usr/local
  # clone openpose
  !git clone -q --depth 1 $git_repo_url
  !sed -i 's/execute_process(COMMAND git checkout master WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/execute_process(COMMAND git checkout f019d0dfe86f49d1140961f8c7dec22130c83154 WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/g' openpose/CMakeLists.txt
  # install system dependencies
  !apt-get -qq install -y libatlas-base-dev libprotobuf-dev libleveldb-dev libsnappy-dev libhdf5-serial-dev protobuf-compiler libgflags-dev libgoogle-glog-dev liblmdb-dev opencl-headers ocl-icd-opencl-dev libviennacl-dev
  # install python dependencies
  !pip install -q youtube-dl
  # build openpose
  !cd openpose && rm -rf build || true && mkdir build && cd build && cmake .. && make -j`nproc`
  

Selecting previously unselected package libgflags2.2.
(Reading database ... 144568 files and directories currently installed.)
Preparing to unpack .../00-libgflags2.2_2.2.1-1_amd64.deb ...
Unpacking libgflags2.2 (2.2.1-1) ...
Selecting previously unselected package libgflags-dev.
Preparing to unpack .../01-libgflags-dev_2.2.1-1_amd64.deb ...
Unpacking libgflags-dev (2.2.1-1) ...
Selecting previously unselected package libgoogle-glog0v5.
Preparing to unpack .../02-libgoogle-glog0v5_0.3.5-1_amd64.deb ...
Unpacking libgoogle-glog0v5 (0.3.5-1) ...
Selecting previously unselected package libgoogle-glog-dev.
Preparing to unpack .../03-libgoogle-glog-dev_0.3.5-1_amd64.deb ...
Unpacking libgoogle-glog-dev (0.3.5-1) ...
Selecting previously unselected package libhdf5-serial-dev.
Preparing to unpack .../04-libhdf5-serial-dev_1.10.0-patch1+docs-4_all.deb ...
Unpacking libhdf5-serial-dev (1.10.0-patch1+docs-4) ...
Selecting previously unselected package libleveldb1v5:amd64.
Preparing to unpack ...

In [7]:
%%shell
for i in {1..80}
{
  mkdir -p ./drive/My\ Drive/636/data/part8_json/pos/$i
}

In [9]:
# detect poses of images
%%shell
cd openpose
for i in {1..80}
{ 
   ./build/examples/openpose/openpose.bin --image_dir ../drive/My\ Drive/636/data/part8_data_refine_part4/pos/frames$i --write_json ../drive/My\ Drive/636/data/part8_json/pos/$i --number_people_max 1 --keypoint_scale 3 --render_pose 0 --display 0 --face
}

Starting OpenPose demo...
Configuring OpenPose...
Starting thread(s)...
Auto-detecting all available GPUs... Detected 1 GPU(s), using 1 of them starting at GPU 0.
OpenPose demo successfully finished. Total time: 438.018238 seconds.
Starting OpenPose demo...
Configuring OpenPose...
Starting thread(s)...
Auto-detecting all available GPUs... Detected 1 GPU(s), using 1 of them starting at GPU 0.
OpenPose demo successfully finished. Total time: 262.206976 seconds.
Starting OpenPose demo...
Configuring OpenPose...
Starting thread(s)...
Auto-detecting all available GPUs... Detected 1 GPU(s), using 1 of them starting at GPU 0.
OpenPose demo successfully finished. Total time: 44.819807 seconds.
Starting OpenPose demo...
Configuring OpenPose...
Starting thread(s)...
Auto-detecting all available GPUs... Detected 1 GPU(s), using 1 of them starting at GPU 0.
OpenPose demo successfully finished. Total time: 40.368102 seconds.
Starting OpenPose demo...
Configuring OpenPose...
Starting thread(s)...
Au

In [10]:
pwd

'/content'

In [11]:
%%shell
for i in {1..91}
{
  mkdir -p ./drive/My\ Drive/636/data/part8_json/neg/$i
}

In [12]:
# detect poses of images
%%shell
cd openpose
for i in {1..91}
{  
   ./build/examples/openpose/openpose.bin --image_dir ../drive/My\ Drive/636/data/part8_data_refine_part4/neg/$i --write_json ../drive/My\ Drive/636/data/part8_json/neg/$i --number_people_max 1 --keypoint_scale 3 --render_pose 0 --display 0 --face
}

Starting OpenPose demo...
Configuring OpenPose...
Starting thread(s)...
Auto-detecting all available GPUs... Detected 1 GPU(s), using 1 of them starting at GPU 0.
OpenPose demo successfully finished. Total time: 44.939951 seconds.
Starting OpenPose demo...
Configuring OpenPose...
Starting thread(s)...
Auto-detecting all available GPUs... Detected 1 GPU(s), using 1 of them starting at GPU 0.
OpenPose demo successfully finished. Total time: 44.026581 seconds.
Starting OpenPose demo...
Configuring OpenPose...
Starting thread(s)...
Auto-detecting all available GPUs... Detected 1 GPU(s), using 1 of them starting at GPU 0.
OpenPose demo successfully finished. Total time: 61.175276 seconds.
Starting OpenPose demo...
Configuring OpenPose...
Starting thread(s)...
Auto-detecting all available GPUs... Detected 1 GPU(s), using 1 of them starting at GPU 0.
OpenPose demo successfully finished. Total time: 58.932172 seconds.
Starting OpenPose demo...
Configuring OpenPose...
Starting thread(s)...
Auto

In [69]:
# detect poses of images
%%shell
cd openpose
for i in {326..335}
{  
  mkdir ../drive/My\ Drive/636/data/part8_json/$i
  ./build/examples/openpose/openpose.bin --image_dir ../drive/My\ Drive/636/data/part8_data_refine_part4/$i --write_json ../drive/My\ Drive/636/data/part8_json/$i --number_people_max 1 --keypoint_scale 3 --render_pose 0 --display 0 --face --write_video_fps 30 --write_video ../drive/My\ Drive/636/data/part8_json/$i.mp4
}

mkdir: cannot create directory ‘../drive/My Drive/636/data/part8_json/326’: File exists
Starting OpenPose demo...
Configuring OpenPose...
Starting thread(s)...
In order to save the rendered frames (`--write_images` or `--write_video`), you cannot disable `--render_pose`.
Auto-detecting all available GPUs... Detected 1 GPU(s), using 1 of them starting at GPU 0.
ffmpeg version 3.4.6-0ubuntu0.18.04.1 Copyright (c) 2000-2019 the FFmpeg developers
built with gcc 7 (Ubuntu 7.3.0-16ubuntu3)
configuration: --prefix=/usr --extra-version=0ubuntu0.18.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-lib

In [0]:
def show_local_mp4_video(file_name, width=640, height=480):
  import io
  import base64
  from IPython.display import HTML
  video_encoded = base64.b64encode(io.open(file_name, 'rb').read())
  return HTML(data='''<video width="{0}" height="{1}" alt="test" controls>
                        <source src="data:video/mp4;base64,{2}" type="video/mp4" />
                      </video>'''.format(width, height, video_encoded.decode('ascii')))

In [0]:
# show_local_mp4_video('./drive/My Drive/636/data/part6_json/29o.mp4')

Read input video (Change **FILEPATH** to the video you are going to use)

In [15]:
%%shell
cd '/content/drive/My Drive/636/data/part8_json/pos/1'
for i in {1..80}
{
  cd ../$i
  echo $i && ls -1 | grep -v ^l | wc -l
}

1
314
2
193
3
28
4
26
5
28
6
30
7
17
8
16
9
23
10
34
11
11
12
11
13
11
14
13
15
38
16
27
17
16
18
14
19
24
20
24
21
55
22
16
23
13
24
28
25
188
26
18
27
19
28
15
29
12
30
13
31
22
32
56
33
13
34
11
35
16
36
41
37
23
38
24
39
11
40
12
41
14
42
36
43
37
44
11
45
7
46
8
47
11
48
35
49
27
50
13
51
10
52
9
53
8
54
8
55
38
56
12
57
12
58
23
59
13
60
11
61
9
62
23
63
19
64
49
65
18
66
14
67
21
68
9
69
29
70
17
71
39
72
31
73
40
74
89
75
33
76
48
77
77
78
134
79
79
80
26


In [0]:
import json
import pandas as pd
import numpy as np
import os, os.path
import time
import csv

N_FACE = 70
N_POSE = 8
FOLDER = './drive/My Drive/636/data/part8_json/'
CSV_FOLDER = './drive/My Drive/636/data/part8_csv/'
COL_NAMES = ['vid', 'frame', 'label']
COL_NAMES += ['f_' + str(i) for i in range(N_FACE * 2)]
COL_NAMES += ['p_' + str(i) for i in range(N_POSE * 2)]

droplist_f = set([(i * 3 + 2) for i in range(N_FACE)])
droplist_p = set([(i * 3 + 2) for i in range(N_POSE)])
idx_f = [i for i in range(N_FACE * 3) if (i not in droplist_f)]
idx_p = [i for i in range(N_POSE * 3) if (i not in droplist_p)]

In [0]:
# collect the json output of openpose and combine them into csv
def pos_json_to_csv(folderpath, csv_path, vid, write_file):
  idx = 0
  csv_writer = csv.writer(write_file)
  for name in sorted(os.listdir(folderpath)): 
    if name[-4:] == 'json':
      filepath = os.path.join(folderpath, name)   
      if os.path.isfile(filepath):
        with open(filepath, 'r') as f:
          data = json.load(f)['people']
          if len(data) > 0:
            print(filepath)
            print(data)
            row = [vid, name, 1]
            row_face = [float(data[0]['face_keypoints_2d'][j]) for j in idx_f]
            row_pose = [float(data[0]['pose_keypoints_2d'][j]) for j in idx_p]
            row += row_face + row_pose
            row = np.array(row)
            csv_writer.writerow(row)
      idx += 1
  print(vid, idx)

def neg_json_to_csv(folderpath, csv_path, vid, write_file):
  idx = 0
  csv_writer = csv.writer(write_file)
  for name in sorted(os.listdir(folderpath)): 
    if name[-4:] == 'json':
      filepath = os.path.join(folderpath, name)
      if os.path.isfile(filepath):
        with open(filepath, 'r') as f:
          data = json.load(f)['people']
          if len(data) > 0:
            row = [vid, name, 0]
            row_face = [float(data[0]['face_keypoints_2d'][j]) for j in idx_f]
            row_pose = [float(data[0]['pose_keypoints_2d'][j]) for j in idx_p]
            row += row_face + row_pose
            row = np.array(row)
            csv_writer.writerow(row)
      idx += 1
  print(vid, idx)

In [0]:
rm -rf './drive/My Drive/636/data/part8_csv/pos.csv'
rm -rf './drive/My Drive/636/data/part8_csv/pos_.csv'
rm -rf './drive/My Drive/636/data/part8_csv/neg.csv'
rm -rf './drive/My Drive/636/data/part8_csv/neg_.csv'

In [49]:
pos_csv_path = CSV_FOLDER + 'pos.csv'
pos_write_file = open(pos_csv_path, 'w')
csv_writer = csv.writer(pos_write_file) 
csv_writer.writerow(COL_NAMES)
for i in range(1, 81):
  folderpath = FOLDER + "pos/" + str(i) + '/' 
  pos_json_to_csv(folderpath, pos_csv_path, i, pos_write_file)

1 314
2 193
3 28
4 26
5 28
6 30
7 17
8 16
9 23
10 34
11 11
12 11
13 11
14 13
15 38
16 27
17 16
18 14
19 24
20 24
21 55
22 16
23 13
24 28
25 188
26 18
27 19
28 15
29 12
30 13
31 22
32 56
33 13
34 11
35 16
36 41
37 23
38 24
39 11
40 12
41 14
42 36
43 37
44 11
45 7
46 8
47 11
48 35
49 27
50 13
51 10
52 9
53 8
54 8
55 38
56 12
57 12
58 23
59 13
60 11
61 9
62 23
63 19
64 49
65 18
66 14
67 21
68 9
69 29
70 17
71 39
72 31
73 40
74 89
75 33
76 48
77 77
78 134
79 79
80 26


In [50]:
neg_csv_path = CSV_FOLDER + 'neg.csv'
neg_write_file = open(neg_csv_path, 'w')
csv_writer = csv.writer(neg_write_file) 
csv_writer.writerow(COL_NAMES) 
for i in range(1, 92):
  folderpath = FOLDER + "neg/" + str(i) + '/' 
  neg_json_to_csv(folderpath, neg_csv_path, i, neg_write_file)

1 48
2 48
3 72
4 72
5 144
6 72
7 72
8 72
9 72
10 72
11 96
12 48
13 72
14 72
15 72
16 72
17 72
18 48
19 48
20 48
21 72
22 72
23 72
24 72
25 72
26 72
27 72
28 72
29 72
30 72
31 96
32 48
33 72
34 96
35 96
36 72
37 48
38 24
39 24
40 24
41 24
42 72
43 24
44 24
45 25
46 48
47 24
48 24
49 24
50 48
51 24
52 24
53 95
54 48
55 48
56 9
57 48
58 24
59 48
60 24
61 24
62 48
63 24
64 21
65 12
66 24
67 8
68 20
69 9
70 21
71 24
72 48
73 24
74 24
75 15
76 24
77 24
78 24
79 24
80 24
81 24
82 24
83 24
84 24
85 24
86 71
87 15
88 12
89 16
90 15
91 9


In [33]:
pwd

'/content'

In [0]:
vid = []
num = []
cnt = 0
total_sum = 0
with open("cnt.txt", "r") as f:
  lines = f.readlines()
  for line in lines:
    if cnt % 2 == 0:
      vid.append(int(line.rstrip()))
    else:
      tmp = int(line.rstrip())
      num.append(tmp)
      total_sum += tmp
    cnt += 1
cnt_dict = dict(zip(vid, num))

In [36]:
total_sum

2651

Input your own json folder path and your label csv file path here to './drive/My Drive/636/test/'

In [0]:
pos_data = pd.read_csv(CSV_FOLDER + 'pos_.csv')

In [54]:
pos_data.shape

(2660, 159)

In [55]:
pos_data.head(100)

,vid,frame,label,f_0,f_1,f_2,f_3,f_4,f_5,f_6,f_7,f_8,f_9,f_10,f_11,f_12,f_13,f_14,f_15,f_16,f_17,f_18,f_19,f_20,f_21,f_22,f_23,f_24,f_25,f_26,f_27,f_28,f_29,f_30,f_31,f_32,f_33,f_34,f_35,f_36,...,f_116,f_117,f_118,f_119,f_120,f_121,f_122,f_123,f_124,f_125,f_126,f_127,f_128,f_129,f_130,f_131,f_132,f_133,f_134,f_135,f_136,f_137,f_138,f_139,p_0,p_1,p_2,p_3,p_4,p_5,p_6,p_7,p_8,p_9,p_10,p_11,p_12,p_13,p_14,p_15
0,1,100_keypoints.json,1,0.390975,0.431474,0.393926,0.476964,0.400813,0.522453,0.406715,0.566193,0.415569,0.608183,0.42639,0.648423,0.443114,0.679916,0.46574,0.70616,0.490333,0.711409,0.513943,0.699161,0.530667,0.667669,0.544439,0.634427,0.559196,0.594186,0.567065,0.552196,0.572968,0.508456,0.575919,0.457718,0.575919,0.412229,0.403764,0.410479,0.417536,...,0.474593,0.641425,0.460821,0.625679,0.453935,0.604684,0.474593,0.611682,0.487382,0.615181,0.49722,0.609932,0.519846,0.597685,0.49722,0.62043,0.487382,0.623929,0.474593,0.62218,0.440162,0.44897,0.526732,0.454219,0.484881,0.557048,0.494107,0.93061,0.287051,0.96064,0.0,0.0,0.0,0.0,0.708851,0.900612,0.0,0.0,0.0,0.0
1,1,101_keypoints.json,1,0.390488,0.428434,0.394423,0.473923,0.401309,0.522912,0.407212,0.566651,0.416066,0.608641,0.426887,0.648881,0.44361,0.680374,0.466236,0.706618,0.49083,0.710117,0.513456,0.69787,0.530179,0.668127,0.544936,0.634885,0.559692,0.594644,0.568545,0.552654,0.574448,0.507165,0.576415,0.458177,0.576415,0.410938,0.404261,0.410938,0.418033,...,0.47509,0.640133,0.461318,0.626137,0.454431,0.605142,0.47509,0.61214,0.487879,0.61389,0.497716,0.610391,0.519358,0.598144,0.497716,0.620888,0.487879,0.624387,0.47509,0.622638,0.440659,0.44768,0.528212,0.454678,0.484907,0.557043,0.49257,0.933344,0.285526,0.957921,0.0,0.0,0.0,0.0,0.707324,0.903352,0.0,0.0,0.0,0.0
2,1,102_keypoints.json,1,0.391238,0.426875,0.396097,0.471808,0.400955,0.518469,0.406786,0.56513,0.416503,0.608334,0.428163,0.646354,0.444683,0.67919,0.467032,0.705113,0.491325,0.710297,0.514646,0.699928,0.531165,0.668821,0.544769,0.635985,0.560316,0.596237,0.56809,0.554761,0.574892,0.509828,0.575864,0.461438,0.577807,0.413049,0.404842,0.411321,0.419418,...,0.474806,0.639442,0.462173,0.625616,0.4544,0.60315,0.474806,0.611791,0.487438,0.615247,0.498127,0.610063,0.520476,0.597965,0.498127,0.620432,0.48841,0.623888,0.474806,0.62216,0.442739,0.445885,0.52825,0.454526,0.484957,0.55965,0.492585,0.930652,0.287089,0.955171,0.0,0.0,0.0,0.0,0.707302,0.903355,0.0,0.0,0.0,0.0
3,1,103_keypoints.json,1,0.391814,0.420514,0.395739,0.471125,0.401626,0.514755,0.407514,0.561876,0.417327,0.607252,0.429102,0.645647,0.445784,0.680551,0.468354,0.706729,0.492886,0.71022,0.515456,0.696258,0.532138,0.668334,0.546857,0.631685,0.561576,0.596781,0.569426,0.554895,0.575314,0.50952,0.578258,0.460654,0.579239,0.413533,0.406533,0.408297,0.420271,...,0.476204,0.640411,0.462466,0.626449,0.455597,0.602016,0.476204,0.612487,0.488961,0.612487,0.499755,0.610742,0.521343,0.598526,0.499755,0.617723,0.489942,0.624704,0.477186,0.619468,0.443822,0.446692,0.530175,0.455418,0.48645,0.557048,0.492596,0.933332,0.288601,0.952462,0.0,0.0,0.0,0.0,0.708836,0.906046,0.0,0.0,0.0,0.0
4,1,104_keypoints.json,1,0.39203,0.417644,0.396928,0.469907,0.402806,0.513461,0.407703,0.56224,0.417499,0.605794,0.430233,0.644121,0.445906,0.678963,0.468435,0.705095,0.492924,0.70858,0.516434,0.696385,0.532106,0.666768,0.547779,0.631926,0.562472,0.595341,0.570309,0.55353,0.577166,0.506492,0.579125,0.459455,0.580104,0.412417,0.407703,0.408933,0.421417,...,0.477251,0.637152,0.463538,0.623215,0.456681,0.60231,0.477251,0.61102,0.489006,0.612762,0.499781,0.61102,0.522311,0.597083,0.500761,0.614504,0.489986,0.621473,0.477251,0.614504,0.444926,0.445518,0.531127,0.454228,0.487974,0.556981,0.494113,0.933345,0.285535,0.966072,0.0,0.0,0.0,0.0,0.707321,0.906051,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,

In [0]:
pos_data.drop(pos_data.index[pos_data['vid'] == 'vid'], inplace=True)

In [66]:
pos_data.loc[pos_data['vid'] == '80']

,vid,frame,label,f_0,f_1,f_2,f_3,f_4,f_5,f_6,f_7,f_8,f_9,f_10,f_11,f_12,f_13,f_14,f_15,f_16,f_17,f_18,f_19,f_20,f_21,f_22,f_23,f_24,f_25,f_26,f_27,f_28,f_29,f_30,f_31,f_32,f_33,f_34,f_35,f_36,...,f_116,f_117,f_118,f_119,f_120,f_121,f_122,f_123,f_124,f_125,f_126,f_127,f_128,f_129,f_130,f_131,f_132,f_133,f_134,f_135,f_136,f_137,f_138,f_139,p_0,p_1,p_2,p_3,p_4,p_5,p_6,p_7,p_8,p_9,p_10,p_11,p_12,p_13,p_14,p_15


In [47]:
neg_data.vid.unique()

array(['1', 'vid', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11',
       '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22',
       '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33',
       '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44',
       '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55',
       '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66',
       '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77',
       '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88'],
      dtype=object)

In [0]:
neg_data = pd.read_csv(CSV_FOLDER + 'neg.csv')

In [41]:
neg_data

,vid,frame,label,f_0,f_1,f_2,f_3,f_4,f_5,f_6,f_7,f_8,f_9,f_10,f_11,f_12,f_13,f_14,f_15,f_16,f_17,f_18,f_19,f_20,f_21,f_22,f_23,f_24,f_25,f_26,f_27,f_28,f_29,f_30,f_31,f_32,f_33,f_34,f_35,f_36,...,f_116,f_117,f_118,f_119,f_120,f_121,f_122,f_123,f_124,f_125,f_126,f_127,f_128,f_129,f_130,f_131,f_132,f_133,f_134,f_135,f_136,f_137,f_138,f_139,p_0,p_1,p_2,p_3,p_4,p_5,p_6,p_7,p_8,p_9,p_10,p_11,p_12,p_13,p_14,p_15
0,1,10_keypoints.json,-1,0.407031,0.271564,0.407031,0.316707,0.407031,0.358378,0.411902,0.400048,0.419696,0.441719,0.431388,0.472971,0.449899,0.499015,0.470358,0.518114,0.493741,0.525059,0.516149,0.523323,0.535634,0.504224,0.554146,0.481653,0.566811,0.446927,0.575579,0.413938,0.579476,0.374004,0.580451,0.337543,0.582399,0.301081,0.433336,0.235103,0.446002,...,0.483998,0.445191,0.470358,0.438246,0.465487,0.417411,0.484972,0.420883,0.493741,0.426092,0.507381,0.424356,0.523943,0.426092,0.507381,0.426092,0.493741,0.426092,0.484972,0.42262,0.46159,0.275037,0.539531,0.288927,0.497171,0.342492,0.492541,0.731005,0.315692,0.741935,0.0,0.0,0.0,0.0,0.675541,0.706593,0.0,0.0,0.0,0.0
1,1,11_keypoints.json,-1,0.406305,0.271411,0.406305,0.316433,0.407277,0.357992,0.412135,0.39955,0.419909,0.439377,0.431568,0.472278,0.449058,0.499984,0.470435,0.519032,0.493755,0.524227,0.516103,0.522495,0.534564,0.505179,0.553997,0.480936,0.566629,0.448036,0.575374,0.413403,0.57926,0.373576,0.580232,0.337212,0.582175,0.300848,0.433512,0.235047,0.446143,...,0.484038,0.446304,0.470435,0.439377,0.465576,0.416866,0.48501,0.42033,0.493755,0.425525,0.506386,0.425525,0.523876,0.425525,0.507358,0.425525,0.493755,0.425525,0.48501,0.422061,0.46169,0.274874,0.539422,0.288727,0.497187,0.342475,0.492549,0.728391,0.317198,0.739215,0.0,0.0,0.0,0.0,0.67553,0.706579,0.0,0.0,0.0,0.0
2,1,12_keypoints.json,-1,0.406815,0.272844,0.406815,0.316256,0.407789,0.359667,0.412661,0.401343,0.420456,0.441281,0.433123,0.474274,0.452611,0.500321,0.471124,0.519423,0.49451,0.524632,0.51692,0.521159,0.535434,0.502058,0.553947,0.48122,0.565639,0.446491,0.575383,0.413498,0.580255,0.373559,0.58123,0.337093,0.583178,0.300627,0.433123,0.234641,0.446765,...,0.484766,0.444754,0.471124,0.437808,0.466252,0.416971,0.48574,0.418707,0.49451,0.425653,0.507177,0.423917,0.524715,0.425653,0.507177,0.425653,0.49451,0.425653,0.48574,0.420444,0.462355,0.27458,0.540306,0.288472,0.498669,0.342423,0.494087,0.728391,0.317202,0.739226,0.0,0.0,0.0,0.0,0.677044,0.70659,0.0,0.0,0.0,0.0
3,1,13_keypoints.json,-1,0.406553,0.272231,0.406553,0.315886,0.407533,0.357796,0.412432,0.399705,0.421251,0.439868,0.43301,0.4713,0.450647,0.49924,0.471224,0.518448,0.494741,0.525433,0.517277,0.521941,0.534915,0.502732,0.554512,0.481778,0.56529,0.446853,0.574109,0.413675,0.579988,0.373512,0.580968,0.338587,0.581948,0.30017,0.43301,0.23556,0.446727,...,0.483962,0.445107,0.471224,0.438122,0.467305,0.417167,0.485922,0.418914,0.494741,0.424152,0.507479,0.424152,0.525116,0.425898,0.507479,0.424152,0.494741,0.425898,0.485922,0.42066,0.462405,0.273977,0.539814,0.287947,0.498652,0.342416,0.494081,0.731017,0.318727,0.744613,0.0,0.0,0.0,0.0,0.677045,0.706605,0.0,0.0,0.0,0.0
4,1,14_keypoints.json,-1,0.407876,0.272948,0.406903,0.316315,0.407876,0.357948,0.412743,0.399581,0.420531,0.43948,0.433185,0.470705,0.450706,0.49846,0.471147,0.517542,0.494509,0.524481,0.516897,0.522746,0.535391,0.503664,0.554859,0.481113,0.56654,0.448153,0.5753,0.413459,0.579194,0.375295,0.581141,0.338867,0.582114,0.300703,0.433185,0.234784,0.446812,...,0.484775,0.444684,0.471147,0.437745,0.467254,0.416928,0.485748,0.418663,0.494509,0.423867,0.508136,0.423867,0.524684,0.425602,0.508136,0.425602,0.495482,0.425602,0.485748,0.420398,0.462387,0.274682,0.540258,0.28856,0.498671,0.342431,0.49256,0.731067,0.318731,0.739219,0.0,0.0,0.0,0.0,0.677039,0.706594,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

In [0]:
raw_data.drop(raw_data.index[raw_data['vid'] == 'vid'], inplace = True)

In [0]:
raw_data.reset_index(drop=True)

,vid,frame,label,f_0,f_1,f_2,f_3,f_4,f_5,f_6,f_7,f_8,f_9,f_10,f_11,f_12,f_13,f_14,f_15,f_16,f_17,f_18,f_19,f_20,f_21,f_22,f_23,f_24,f_25,f_26,f_27,f_28,f_29,f_30,f_31,f_32,f_33,f_34,f_35,f_36,...,f_116,f_117,f_118,f_119,f_120,f_121,f_122,f_123,f_124,f_125,f_126,f_127,f_128,f_129,f_130,f_131,f_132,f_133,f_134,f_135,f_136,f_137,f_138,f_139,p_0,p_1,p_2,p_3,p_4,p_5,p_6,p_7,p_8,p_9,p_10,p_11,p_12,p_13,p_14,p_15
0,111,111_000000000000_keypoints.json,1,0.304436,0.628547,0.424224,0.281957,0.306717,0.39546,0.306717,0.519097,0.306717,0.604225,0.327252,0.642735,0.354633,0.675164,0.546294,0.606251,0.405971,0.675164,0.609041,0.409648,0.497238,0.675164,0.536027,0.675164,0.557703,0.622466,0.574815,0.563688,0.593069,0.506936,0.606759,0.444104,0.611322,0.381272,0.388858,0.182641,0.41852,...,0.481266,0.654896,0.455027,0.62652,0.442478,0.592064,0.489252,0.610305,0.505224,0.620439,0.517773,0.622466,0.525759,0.628547,0.516632,0.622466,0.504083,0.622466,0.489252,0.616386,0.441337,0.27993,0.58052,0.342762,0.520471,0.423402,0.285857,0.813034,0.14782,0.802122,0.0,0.0,0.0,0.0,0.419295,0.913852,0.0,0.0,0.0,0.0
1,111,111_000000000001_keypoints.json,1,0.347309,0.214129,0.347309,0.214129,0.30237,0.394282,0.30237,0.496641,0.300065,0.60719,0.325416,0.650181,0.354223,0.678841,0.409533,0.678841,0.479824,0.678841,0.608881,0.406565,0.497108,0.678841,0.536286,0.678841,0.560484,0.619473,0.576617,0.560104,0.596206,0.50483,0.607729,0.443414,0.611186,0.379951,0.388792,0.183421,0.419904,...,0.478671,0.656322,0.453321,0.627661,0.441798,0.592859,0.48789,0.611284,0.505174,0.623567,0.515545,0.623567,0.524763,0.627661,0.515545,0.623567,0.504022,0.623567,0.48789,0.613331,0.440646,0.279639,0.577769,0.341055,0.51898,0.428901,0.282778,0.796686,0.141691,0.796663,0.0,0.0,0.0,0.0,0.419276,0.916572,0.0,0.0,0.0,0.0
2,111,111_000000000002_keypoints.json,1,0.304425,0.628224,0.422798,0.283163,0.304425,0.381169,0.304425,0.507759,0.304425,0.609848,0.323962,0.652725,0.351544,0.679268,0.545767,0.609848,0.40441,0.679268,0.608976,0.411795,0.497499,0.679268,0.535424,0.679268,0.55726,0.626182,0.574498,0.56697,0.592886,0.509801,0.606677,0.446506,0.610125,0.381169,0.387171,0.185158,0.415902,...,0.48026,0.65885,0.453827,0.630265,0.440036,0.593513,0.488305,0.613931,0.504394,0.62414,0.517036,0.626182,0.52623,0.630265,0.515887,0.626182,0.503245,0.626182,0.488305,0.618015,0.437738,0.281122,0.575648,0.342375,0.51895,0.445235,0.279717,0.788509,0.150878,0.791206,0.0,0.0,0.0,0.0,0.417758,0.916585,0.0,0.0,0.0,0.0
3,111,111_000000000003_keypoints.json,1,0.304477,0.630774,0.422831,0.283726,0.304477,0.381716,0.304477,0.510328,0.304477,0.606276,0.324011,0.649147,0.351588,0.68181,0.553824,0.612401,0.407893,0.68181,0.60898,0.402131,0.498669,0.68181,0.537737,0.66752,0.55957,0.624649,0.575657,0.563406,0.59634,0.506245,0.60898,0.445001,0.611278,0.381716,0.38606,0.185737,0.415936,...,0.480284,0.659354,0.453855,0.630774,0.442365,0.596069,0.489477,0.614442,0.505563,0.626691,0.517054,0.626691,0.525098,0.630774,0.517054,0.626691,0.504414,0.626691,0.489477,0.618525,0.440067,0.283726,0.575657,0.342929,0.522028,0.445276,0.301187,0.799408,0.150882,0.774929,0.0,0.0,0.0,0.0,0.414691,0.911131,0.0,0.0,0.0,0.0
4,111,111_000000000004_keypoints.json,1,0.308436,0.638,0.318619,0.123412,0.308436,0.392767,0.308436,0.515383,0.308436,0.609858,0.326539,0.652071,0.354825,0.676192,0.397819,0.676192,0.408002,0.676192,0.608264,0.398797,0.49399,0.676192,0.540378,0.660111,0.560744,0.621919,0.579978,0.559606,0.598081,0.503323,0.608264,0.443019,0.61279,0.380706,0.386505,0.187735,0.418185,...,0.480413,0.658101,0.45439,0.63398,0.443076,0.601818,0.488333,0.619909,0.504173,0.623929,0.51775,0.625939,0.524539,0.63398,0.516618,0.627949,0.503041,0.625939,0.488333,0.621919,0.43855,0.284221,0.576584,0.344524,0.520474,0.445237,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

In [0]:
raw_data.vid.unique()

array(['111', '109', '105', '103', '99', '97', '87', '86', '84', '68',
       '64', '58', '57', '47', '43', '41', '40', '38', '35', '34', '33',
       '32', '31', '30', '29', '28', '113', '112', '107', '106', '104',
       '102', '100', '98', '96', '95', '92', '69', '67', '66', '65', '63',
       '62', '61', '60', '59', '53', '52', '50', '48', '39', '37', '36'],
      dtype=object)

In [0]:
raw_data.shape

(23642, 159)

In [0]:
drop_idx = list(raw_data.index[raw_data['vid'] == '33'][-10:])

In [0]:
raw_data.drop(raw_data.index[drop_idx], inplace=True)

In [0]:
# raw_data.loc[raw_data['vid'] == '33']

,index,vid,frame,label,f_0,f_1,f_2,f_3,f_4,f_5,f_6,f_7,f_8,f_9,f_10,f_11,f_12,f_13,f_14,f_15,f_16,f_17,f_18,f_19,f_20,f_21,f_22,f_23,f_24,f_25,f_26,f_27,f_28,f_29,f_30,f_31,f_32,f_33,f_34,f_35,...,f_116,f_117,f_118,f_119,f_120,f_121,f_122,f_123,f_124,f_125,f_126,f_127,f_128,f_129,f_130,f_131,f_132,f_133,f_134,f_135,f_136,f_137,f_138,f_139,p_0,p_1,p_2,p_3,p_4,p_5,p_6,p_7,p_8,p_9,p_10,p_11,p_12,p_13,p_14,p_15
14576,14596,33,33_000000000000_keypoints.json,1,427.557,326.194,429.363,382.166,442.002,436.332,456.446,485.081,474.501,537.441,501.584,578.969,532.278,615.079,562.972,660.217,608.11,660.217,669.498,660.217,725.47,642.162,774.219,609.663,817.552,569.941,850.051,522.997,873.523,461.609,878.94,398.416,880.745,338.833,445.613,237.724,...,573.805,582.58,555.75,569.941,541.306,550.08,575.611,553.691,599.083,553.691,618.943,553.691,673.109,553.691,618.943,553.691,599.083,553.691,575.611,553.691,512.417,333.417,727.275,337.028,592.185,435.396,0.0,0.0,304.147,715.595,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14577,14597,33,33_000000000001_keypoints.json,1,427.56,326.204,429.366,382.174,442.004,436.339,456.448,485.087,474.503,537.446,501.585,578.972,532.278,615.082,562.971,660.219,608.109,660.219,671.3,660.219,727.271,640.359,774.213,609.665,817.545,569.945,853.654,517.586,873.515,458.005,877.126,396.618,878.931,337.037,445.615,237.736,...,573.804,582.583,555.75,569.945,541.306,550.085,575.61,553.695,599.081,553.695,618.941,553.695,673.106,553.695,618.941,553.695,599.081,553.695,575.61,553.695,512.418,333.426,727.271,337.037,592.185,435.399,0.0,0.0,300.247,715.591,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14578,14598,33,33_000000000002_keypoints.json,1,428.289,326.05,430.078,379.739,442.606,435.218,456.923,485.328,474.819,535.439,499.874,576.6,530.298,614.183,560.722,655.345,609.043,655.345,669.891,655.345,727.159,637.448,779.059,607.024,823.8,565.862,856.014,517.542,872.121,454.904,882.859,394.056,882.859,334.998,444.395,236.568,...,575.039,580.18,555.353,567.652,542.826,547.966,575.039,549.756,598.305,551.545,619.781,549.756,671.68,549.756,619.781,549.756,598.305,551.545,575.039,549.756,512.402,331.419,727.159,334.998,596.044,429.585,0.0,0.0,296.315,715.59,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14579,14599,33,33_000000000003_keypoints.json,1,428.28,326.025,430.07,379.72,442.598,435.205,456.917,485.321,473.026,535.436,499.873,576.602,530.301,614.189,560.728,655.355,609.054,655.355,669.908,655.355,727.183,637.457,775.509,607.03,822.044,565.863,856.051,519.328,872.16,456.683,882.899,395.829,882.899,334.974,446.178,236.533,...,575.047,580.182,555.358,567.653,542.83,547.965,575.047,549.755,598.315,551.545,619.793,549.755,671.698,549.755,619.793,549.755,598.315,551.545,576.836,549.755,512.402,331.394,727.183,334.974,592.202,429.57,0.0,0.0,296.343,715.591,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14580,14600,33,33_000000000004_keypoints.json,1,428.547,323.818,433.723,379.03,444.075,429.066,461.329,482.553,476.857,532.589,504.463,577.449,533.795,613.681,564.852,637.837,607.986,637.837,689.079,637.837,727.037,637.837,778.799,605.055,821.933,567.096,852.99,518.786,875.42,458.397,878.87,398.009,880.596,332.445,447.526,235.824,...,575.204,577.449,554.499,565.371,544.147,544.666,575.204,548.117,599.359,549.842,620.064,548.117,671.825,549.842,618.338,548.117,599.359,549.842,575.204,548.117,514.816,328.994,727.037,330.719,598.036,427.614,0.0,0.0,300.246,715.594,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14861,14881,33,33_000000000285_keypoints.json,1,371.263,221.716,369.269,281.516,377.243,333.342,387.21,397.129,403.156,454.936,433.056,506.762,468.936,556.596,494.85,592.476,532.723,592.476,626.41,592.476,674.25,592.476,744.016,572.

In [0]:
raw_data.reset_index(drop=True)

,vid,frame,label,f_0,f_1,f_2,f_3,f_4,f_5,f_6,f_7,f_8,f_9,f_10,f_11,f_12,f_13,f_14,f_15,f_16,f_17,f_18,f_19,f_20,f_21,f_22,f_23,f_24,f_25,f_26,f_27,f_28,f_29,f_30,f_31,f_32,f_33,f_34,f_35,f_36,...,f_116,f_117,f_118,f_119,f_120,f_121,f_122,f_123,f_124,f_125,f_126,f_127,f_128,f_129,f_130,f_131,f_132,f_133,f_134,f_135,f_136,f_137,f_138,f_139,p_0,p_1,p_2,p_3,p_4,p_5,p_6,p_7,p_8,p_9,p_10,p_11,p_12,p_13,p_14,p_15
0,111,111_000000000000_keypoints.json,1,0.304436,0.628547,0.424224,0.281957,0.306717,0.39546,0.306717,0.519097,0.306717,0.604225,0.327252,0.642735,0.354633,0.675164,0.546294,0.606251,0.405971,0.675164,0.609041,0.409648,0.497238,0.675164,0.536027,0.675164,0.557703,0.622466,0.574815,0.563688,0.593069,0.506936,0.606759,0.444104,0.611322,0.381272,0.388858,0.182641,0.41852,...,0.481266,0.654896,0.455027,0.62652,0.442478,0.592064,0.489252,0.610305,0.505224,0.620439,0.517773,0.622466,0.525759,0.628547,0.516632,0.622466,0.504083,0.622466,0.489252,0.616386,0.441337,0.27993,0.58052,0.342762,0.520471,0.423402,0.285857,0.813034,0.14782,0.802122,0.0,0.0,0.0,0.0,0.419295,0.913852,0.0,0.0,0.0,0.0
1,111,111_000000000001_keypoints.json,1,0.347309,0.214129,0.347309,0.214129,0.30237,0.394282,0.30237,0.496641,0.300065,0.60719,0.325416,0.650181,0.354223,0.678841,0.409533,0.678841,0.479824,0.678841,0.608881,0.406565,0.497108,0.678841,0.536286,0.678841,0.560484,0.619473,0.576617,0.560104,0.596206,0.50483,0.607729,0.443414,0.611186,0.379951,0.388792,0.183421,0.419904,...,0.478671,0.656322,0.453321,0.627661,0.441798,0.592859,0.48789,0.611284,0.505174,0.623567,0.515545,0.623567,0.524763,0.627661,0.515545,0.623567,0.504022,0.623567,0.48789,0.613331,0.440646,0.279639,0.577769,0.341055,0.51898,0.428901,0.282778,0.796686,0.141691,0.796663,0.0,0.0,0.0,0.0,0.419276,0.916572,0.0,0.0,0.0,0.0
2,111,111_000000000002_keypoints.json,1,0.304425,0.628224,0.422798,0.283163,0.304425,0.381169,0.304425,0.507759,0.304425,0.609848,0.323962,0.652725,0.351544,0.679268,0.545767,0.609848,0.40441,0.679268,0.608976,0.411795,0.497499,0.679268,0.535424,0.679268,0.55726,0.626182,0.574498,0.56697,0.592886,0.509801,0.606677,0.446506,0.610125,0.381169,0.387171,0.185158,0.415902,...,0.48026,0.65885,0.453827,0.630265,0.440036,0.593513,0.488305,0.613931,0.504394,0.62414,0.517036,0.626182,0.52623,0.630265,0.515887,0.626182,0.503245,0.626182,0.488305,0.618015,0.437738,0.281122,0.575648,0.342375,0.51895,0.445235,0.279717,0.788509,0.150878,0.791206,0.0,0.0,0.0,0.0,0.417758,0.916585,0.0,0.0,0.0,0.0
3,111,111_000000000003_keypoints.json,1,0.304477,0.630774,0.422831,0.283726,0.304477,0.381716,0.304477,0.510328,0.304477,0.606276,0.324011,0.649147,0.351588,0.68181,0.553824,0.612401,0.407893,0.68181,0.60898,0.402131,0.498669,0.68181,0.537737,0.66752,0.55957,0.624649,0.575657,0.563406,0.59634,0.506245,0.60898,0.445001,0.611278,0.381716,0.38606,0.185737,0.415936,...,0.480284,0.659354,0.453855,0.630774,0.442365,0.596069,0.489477,0.614442,0.505563,0.626691,0.517054,0.626691,0.525098,0.630774,0.517054,0.626691,0.504414,0.626691,0.489477,0.618525,0.440067,0.283726,0.575657,0.342929,0.522028,0.445276,0.301187,0.799408,0.150882,0.774929,0.0,0.0,0.0,0.0,0.414691,0.911131,0.0,0.0,0.0,0.0
4,111,111_000000000004_keypoints.json,1,0.308436,0.638,0.318619,0.123412,0.308436,0.392767,0.308436,0.515383,0.308436,0.609858,0.326539,0.652071,0.354825,0.676192,0.397819,0.676192,0.408002,0.676192,0.608264,0.398797,0.49399,0.676192,0.540378,0.660111,0.560744,0.621919,0.579978,0.559606,0.598081,0.503323,0.608264,0.443019,0.61279,0.380706,0.386505,0.187735,0.418185,...,0.480413,0.658101,0.45439,0.63398,0.443076,0.601818,0.488333,0.619909,0.504173,0.623929,0.51775,0.625939,0.524539,0.63398,0.516618,0.627949,0.503041,0.625939,0.488333,0.621919,0.43855,0.284221,0.576584,0.344524,0.520474,0.445237,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

In [0]:
drop_idx = list(raw_data.index[raw_data['vid'] == '102'][:6])
raw_data.drop(raw_data.index[drop_idx], inplace=True)
raw_data.reset_index(drop=True)

,vid,frame,label,f_0,f_1,f_2,f_3,f_4,f_5,f_6,f_7,f_8,f_9,f_10,f_11,f_12,f_13,f_14,f_15,f_16,f_17,f_18,f_19,f_20,f_21,f_22,f_23,f_24,f_25,f_26,f_27,f_28,f_29,f_30,f_31,f_32,f_33,f_34,f_35,f_36,...,f_116,f_117,f_118,f_119,f_120,f_121,f_122,f_123,f_124,f_125,f_126,f_127,f_128,f_129,f_130,f_131,f_132,f_133,f_134,f_135,f_136,f_137,f_138,f_139,p_0,p_1,p_2,p_3,p_4,p_5,p_6,p_7,p_8,p_9,p_10,p_11,p_12,p_13,p_14,p_15
0,111,111_000000000000_keypoints.json,1,0.304436,0.628547,0.424224,0.281957,0.306717,0.39546,0.306717,0.519097,0.306717,0.604225,0.327252,0.642735,0.354633,0.675164,0.546294,0.606251,0.405971,0.675164,0.609041,0.409648,0.497238,0.675164,0.536027,0.675164,0.557703,0.622466,0.574815,0.563688,0.593069,0.506936,0.606759,0.444104,0.611322,0.381272,0.388858,0.182641,0.41852,...,0.481266,0.654896,0.455027,0.62652,0.442478,0.592064,0.489252,0.610305,0.505224,0.620439,0.517773,0.622466,0.525759,0.628547,0.516632,0.622466,0.504083,0.622466,0.489252,0.616386,0.441337,0.27993,0.58052,0.342762,0.520471,0.423402,0.285857,0.813034,0.14782,0.802122,0.0,0.0,0.0,0.0,0.419295,0.913852,0.0,0.0,0.0,0.0
1,111,111_000000000001_keypoints.json,1,0.347309,0.214129,0.347309,0.214129,0.30237,0.394282,0.30237,0.496641,0.300065,0.60719,0.325416,0.650181,0.354223,0.678841,0.409533,0.678841,0.479824,0.678841,0.608881,0.406565,0.497108,0.678841,0.536286,0.678841,0.560484,0.619473,0.576617,0.560104,0.596206,0.50483,0.607729,0.443414,0.611186,0.379951,0.388792,0.183421,0.419904,...,0.478671,0.656322,0.453321,0.627661,0.441798,0.592859,0.48789,0.611284,0.505174,0.623567,0.515545,0.623567,0.524763,0.627661,0.515545,0.623567,0.504022,0.623567,0.48789,0.613331,0.440646,0.279639,0.577769,0.341055,0.51898,0.428901,0.282778,0.796686,0.141691,0.796663,0.0,0.0,0.0,0.0,0.419276,0.916572,0.0,0.0,0.0,0.0
2,111,111_000000000002_keypoints.json,1,0.304425,0.628224,0.422798,0.283163,0.304425,0.381169,0.304425,0.507759,0.304425,0.609848,0.323962,0.652725,0.351544,0.679268,0.545767,0.609848,0.40441,0.679268,0.608976,0.411795,0.497499,0.679268,0.535424,0.679268,0.55726,0.626182,0.574498,0.56697,0.592886,0.509801,0.606677,0.446506,0.610125,0.381169,0.387171,0.185158,0.415902,...,0.48026,0.65885,0.453827,0.630265,0.440036,0.593513,0.488305,0.613931,0.504394,0.62414,0.517036,0.626182,0.52623,0.630265,0.515887,0.626182,0.503245,0.626182,0.488305,0.618015,0.437738,0.281122,0.575648,0.342375,0.51895,0.445235,0.279717,0.788509,0.150878,0.791206,0.0,0.0,0.0,0.0,0.417758,0.916585,0.0,0.0,0.0,0.0
3,111,111_000000000003_keypoints.json,1,0.304477,0.630774,0.422831,0.283726,0.304477,0.381716,0.304477,0.510328,0.304477,0.606276,0.324011,0.649147,0.351588,0.68181,0.553824,0.612401,0.407893,0.68181,0.60898,0.402131,0.498669,0.68181,0.537737,0.66752,0.55957,0.624649,0.575657,0.563406,0.59634,0.506245,0.60898,0.445001,0.611278,0.381716,0.38606,0.185737,0.415936,...,0.480284,0.659354,0.453855,0.630774,0.442365,0.596069,0.489477,0.614442,0.505563,0.626691,0.517054,0.626691,0.525098,0.630774,0.517054,0.626691,0.504414,0.626691,0.489477,0.618525,0.440067,0.283726,0.575657,0.342929,0.522028,0.445276,0.301187,0.799408,0.150882,0.774929,0.0,0.0,0.0,0.0,0.414691,0.911131,0.0,0.0,0.0,0.0
4,111,111_000000000004_keypoints.json,1,0.308436,0.638,0.318619,0.123412,0.308436,0.392767,0.308436,0.515383,0.308436,0.609858,0.326539,0.652071,0.354825,0.676192,0.397819,0.676192,0.408002,0.676192,0.608264,0.398797,0.49399,0.676192,0.540378,0.660111,0.560744,0.621919,0.579978,0.559606,0.598081,0.503323,0.608264,0.443019,0.61279,0.380706,0.386505,0.187735,0.418185,...,0.480413,0.658101,0.45439,0.63398,0.443076,0.601818,0.488333,0.619909,0.504173,0.623929,0.51775,0.625939,0.524539,0.63398,0.516618,0.627949,0.503041,0.625939,0.488333,0.621919,0.43855,0.284221,0.576584,0.344524,0.520474,0.445237,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

In [0]:
raw_data['label'] = raw_data['label'].astype(int)

In [0]:
cols = list(raw_data.columns)
cols.remove('vid')
cols.remove('frame')
cols.remove('label')
for col in cols:
    raw_data[col] = raw_data[col].astype(float)

In [0]:
raw_data['vid'] = raw_data['vid'].astype(int)

In [0]:
raw_data.drop(raw_data.index[raw_data['f_0'] >= 1], inplace=True)
raw_data.reset_index(drop=True)

,vid,frame,label,f_0,f_1,f_2,f_3,f_4,f_5,f_6,f_7,f_8,f_9,f_10,f_11,f_12,f_13,f_14,f_15,f_16,f_17,f_18,f_19,f_20,f_21,f_22,f_23,f_24,f_25,f_26,f_27,f_28,f_29,f_30,f_31,f_32,f_33,f_34,f_35,f_36,...,f_116,f_117,f_118,f_119,f_120,f_121,f_122,f_123,f_124,f_125,f_126,f_127,f_128,f_129,f_130,f_131,f_132,f_133,f_134,f_135,f_136,f_137,f_138,f_139,p_0,p_1,p_2,p_3,p_4,p_5,p_6,p_7,p_8,p_9,p_10,p_11,p_12,p_13,p_14,p_15
0,111,111_000000000000_keypoints.json,1,0.304436,0.628547,0.424224,0.281957,0.306717,0.395460,0.306717,0.519097,0.306717,0.604225,0.327252,0.642735,0.354633,0.675164,0.546294,0.606251,0.405971,0.675164,0.609041,0.409648,0.497238,0.675164,0.536027,0.675164,0.557703,0.622466,0.574815,0.563688,0.593069,0.506936,0.606759,0.444104,0.611322,0.381272,0.388858,0.182641,0.418520,...,0.481266,0.654896,0.455027,0.626520,0.442478,0.592064,0.489252,0.610305,0.505224,0.620439,0.517773,0.622466,0.525759,0.628547,0.516632,0.622466,0.504083,0.622466,0.489252,0.616386,0.441337,0.279930,0.580520,0.342762,0.520471,0.423402,0.285857,0.813034,0.147820,0.802122,0.000000,0.000000,0.000000,0.000000,0.419295,0.913852,0.000000,0.000000,0.000000,0.000000
1,111,111_000000000001_keypoints.json,1,0.347309,0.214129,0.347309,0.214129,0.302370,0.394282,0.302370,0.496641,0.300065,0.607190,0.325416,0.650181,0.354223,0.678841,0.409533,0.678841,0.479824,0.678841,0.608881,0.406565,0.497108,0.678841,0.536286,0.678841,0.560484,0.619473,0.576617,0.560104,0.596206,0.504830,0.607729,0.443414,0.611186,0.379951,0.388792,0.183421,0.419904,...,0.478671,0.656322,0.453321,0.627661,0.441798,0.592859,0.487890,0.611284,0.505174,0.623567,0.515545,0.623567,0.524763,0.627661,0.515545,0.623567,0.504022,0.623567,0.487890,0.613331,0.440646,0.279639,0.577769,0.341055,0.518980,0.428901,0.282778,0.796686,0.141691,0.796663,0.000000,0.000000,0.000000,0.000000,0.419276,0.916572,0.000000,0.000000,0.000000,0.000000
2,111,111_000000000002_keypoints.json,1,0.304425,0.628224,0.422798,0.283163,0.304425,0.381169,0.304425,0.507759,0.304425,0.609848,0.323962,0.652725,0.351544,0.679268,0.545767,0.609848,0.404410,0.679268,0.608976,0.411795,0.497499,0.679268,0.535424,0.679268,0.557260,0.626182,0.574498,0.566970,0.592886,0.509801,0.606677,0.446506,0.610125,0.381169,0.387171,0.185158,0.415902,...,0.480260,0.658850,0.453827,0.630265,0.440036,0.593513,0.488305,0.613931,0.504394,0.624140,0.517036,0.626182,0.526230,0.630265,0.515887,0.626182,0.503245,0.626182,0.488305,0.618015,0.437738,0.281122,0.575648,0.342375,0.518950,0.445235,0.279717,0.788509,0.150878,0.791206,0.000000,0.000000,0.000000,0.000000,0.417758,0.916585,0.000000,0.000000,0.000000,0.000000
3,111,111_000000000003_keypoints.json,1,0.304477,0.630774,0.422831,0.283726,0.304477,0.381716,0.304477,0.510328,0.304477,0.606276,0.324011,0.649147,0.351588,0.681810,0.553824,0.612401,0.407893,0.681810,0.608980,0.402131,0.498669,0.681810,0.537737,0.667520,0.559570,0.624649,0.575657,0.563406,0.596340,0.506245,0.608980,0.445001,0.611278,0.381716,0.386060,0.185737,0.415936,...,0.480284,0.659354,0.453855,0.630774,0.442365,0.596069,0.489477,0.614442,0.505563,0.626691,0.517054,0.626691,0.525098,0.630774,0.517054,0.626691,0.504414,0.626691,0.489477,0.618525,0.440067,0.283726,0.575657,0.342929,0.522028,0.445276,0.301187,0.799408,0.150882,0.774929,0.000000,0.000000,0.000000,0.000000,0.414691,0.911131,0.000000,0.000000,0.000000,0.000000
4,111,111_000000000004_keypoints.json,1,0.308436,0.638000,0.318619,0.123412,0.308436,0.392767,0.308436,0.515383,0.308436,0.609858,0.326539,0.652071,0.354825,0.676192,0.397819,0.676192,0.408002,0.676192,0.608264,0.398797,0.493990,0.676192,0.540378,0.660111,0.560744,0.621919,0.579978,0.559606,0.598081,0.503323,0.608264,0.443019,0.612790,0.380706,0.386505,0.187735,0.418185,...,0.480413,0.658101,0.454390,0.633980,0.443076,0.601818,0.488333,0.619909,0.504173,0.623929,0.517750,0.625939,0.524539,0.633980,0.516618,0.627949,0.503041,0.625939,0.488333,0.621919,0.438550,0.284221,0.576584,0.344524,0.520474,0.445237,0.000000,0.000000,0.000000,0.00

In [0]:
raw_data

,vid,frame,label,f_0,f_1,f_2,f_3,f_4,f_5,f_6,f_7,f_8,f_9,f_10,f_11,f_12,f_13,f_14,f_15,f_16,f_17,f_18,f_19,f_20,f_21,f_22,f_23,f_24,f_25,f_26,f_27,f_28,f_29,f_30,f_31,f_32,f_33,f_34,f_35,f_36,...,f_116,f_117,f_118,f_119,f_120,f_121,f_122,f_123,f_124,f_125,f_126,f_127,f_128,f_129,f_130,f_131,f_132,f_133,f_134,f_135,f_136,f_137,f_138,f_139,p_0,p_1,p_2,p_3,p_4,p_5,p_6,p_7,p_8,p_9,p_10,p_11,p_12,p_13,p_14,p_15
0,111,111_000000000000_keypoints.json,1,0.304436,0.628547,0.424224,0.281957,0.306717,0.395460,0.306717,0.519097,0.306717,0.604225,0.327252,0.642735,0.354633,0.675164,0.546294,0.606251,0.405971,0.675164,0.609041,0.409648,0.497238,0.675164,0.536027,0.675164,0.557703,0.622466,0.574815,0.563688,0.593069,0.506936,0.606759,0.444104,0.611322,0.381272,0.388858,0.182641,0.418520,...,0.481266,0.654896,0.455027,0.626520,0.442478,0.592064,0.489252,0.610305,0.505224,0.620439,0.517773,0.622466,0.525759,0.628547,0.516632,0.622466,0.504083,0.622466,0.489252,0.616386,0.441337,0.279930,0.580520,0.342762,0.520471,0.423402,0.285857,0.813034,0.147820,0.802122,0.000000,0.000000,0.000000,0.000000,0.419295,0.913852,0.000000,0.000000,0.000000,0.000000
1,111,111_000000000001_keypoints.json,1,0.347309,0.214129,0.347309,0.214129,0.302370,0.394282,0.302370,0.496641,0.300065,0.607190,0.325416,0.650181,0.354223,0.678841,0.409533,0.678841,0.479824,0.678841,0.608881,0.406565,0.497108,0.678841,0.536286,0.678841,0.560484,0.619473,0.576617,0.560104,0.596206,0.504830,0.607729,0.443414,0.611186,0.379951,0.388792,0.183421,0.419904,...,0.478671,0.656322,0.453321,0.627661,0.441798,0.592859,0.487890,0.611284,0.505174,0.623567,0.515545,0.623567,0.524763,0.627661,0.515545,0.623567,0.504022,0.623567,0.487890,0.613331,0.440646,0.279639,0.577769,0.341055,0.518980,0.428901,0.282778,0.796686,0.141691,0.796663,0.000000,0.000000,0.000000,0.000000,0.419276,0.916572,0.000000,0.000000,0.000000,0.000000
2,111,111_000000000002_keypoints.json,1,0.304425,0.628224,0.422798,0.283163,0.304425,0.381169,0.304425,0.507759,0.304425,0.609848,0.323962,0.652725,0.351544,0.679268,0.545767,0.609848,0.404410,0.679268,0.608976,0.411795,0.497499,0.679268,0.535424,0.679268,0.557260,0.626182,0.574498,0.566970,0.592886,0.509801,0.606677,0.446506,0.610125,0.381169,0.387171,0.185158,0.415902,...,0.480260,0.658850,0.453827,0.630265,0.440036,0.593513,0.488305,0.613931,0.504394,0.624140,0.517036,0.626182,0.526230,0.630265,0.515887,0.626182,0.503245,0.626182,0.488305,0.618015,0.437738,0.281122,0.575648,0.342375,0.518950,0.445235,0.279717,0.788509,0.150878,0.791206,0.000000,0.000000,0.000000,0.000000,0.417758,0.916585,0.000000,0.000000,0.000000,0.000000
3,111,111_000000000003_keypoints.json,1,0.304477,0.630774,0.422831,0.283726,0.304477,0.381716,0.304477,0.510328,0.304477,0.606276,0.324011,0.649147,0.351588,0.681810,0.553824,0.612401,0.407893,0.681810,0.608980,0.402131,0.498669,0.681810,0.537737,0.667520,0.559570,0.624649,0.575657,0.563406,0.596340,0.506245,0.608980,0.445001,0.611278,0.381716,0.386060,0.185737,0.415936,...,0.480284,0.659354,0.453855,0.630774,0.442365,0.596069,0.489477,0.614442,0.505563,0.626691,0.517054,0.626691,0.525098,0.630774,0.517054,0.626691,0.504414,0.626691,0.489477,0.618525,0.440067,0.283726,0.575657,0.342929,0.522028,0.445276,0.301187,0.799408,0.150882,0.774929,0.000000,0.000000,0.000000,0.000000,0.414691,0.911131,0.000000,0.000000,0.000000,0.000000
4,111,111_000000000004_keypoints.json,1,0.308436,0.638000,0.318619,0.123412,0.308436,0.392767,0.308436,0.515383,0.308436,0.609858,0.326539,0.652071,0.354825,0.676192,0.397819,0.676192,0.408002,0.676192,0.608264,0.398797,0.493990,0.676192,0.540378,0.660111,0.560744,0.621919,0.579978,0.559606,0.598081,0.503323,0.608264,0.443019,0.612790,0.380706,0.386505,0.187735,0.418185,...,0.480413,0.658101,0.454390,0.633980,0.443076,0.601818,0.488333,0.619909,0.504173,0.623929,0.517750,0.625939,0.524539,0.633980,0.516618,0.627949,0.503041,0.625939,0.488333,0.621919,0.438550,0.284221,0.576584,0.344524,0.520474,0.445237,0.000000,0.000000,0.000000,0.00

In [0]:
raw_data.drop(raw_data.index[raw_data['vid'].isin([36, 37, 38, 39, 40, 41, 42])], inplace=True)
raw_data.reset_index(drop=True)

,vid,frame,label,f_0,f_1,f_2,f_3,f_4,f_5,f_6,f_7,f_8,f_9,f_10,f_11,f_12,f_13,f_14,f_15,f_16,f_17,f_18,f_19,f_20,f_21,f_22,f_23,f_24,f_25,f_26,f_27,f_28,f_29,f_30,f_31,f_32,f_33,f_34,f_35,f_36,...,f_116,f_117,f_118,f_119,f_120,f_121,f_122,f_123,f_124,f_125,f_126,f_127,f_128,f_129,f_130,f_131,f_132,f_133,f_134,f_135,f_136,f_137,f_138,f_139,p_0,p_1,p_2,p_3,p_4,p_5,p_6,p_7,p_8,p_9,p_10,p_11,p_12,p_13,p_14,p_15
0,111,111_000000000000_keypoints.json,1,0.304436,0.628547,0.424224,0.281957,0.306717,0.395460,0.306717,0.519097,0.306717,0.604225,0.327252,0.642735,0.354633,0.675164,0.546294,0.606251,0.405971,0.675164,0.609041,0.409648,0.497238,0.675164,0.536027,0.675164,0.557703,0.622466,0.574815,0.563688,0.593069,0.506936,0.606759,0.444104,0.611322,0.381272,0.388858,0.182641,0.418520,...,0.481266,0.654896,0.455027,0.626520,0.442478,0.592064,0.489252,0.610305,0.505224,0.620439,0.517773,0.622466,0.525759,0.628547,0.516632,0.622466,0.504083,0.622466,0.489252,0.616386,0.441337,0.279930,0.580520,0.342762,0.520471,0.423402,0.285857,0.813034,0.147820,0.802122,0.000000,0.000000,0.000000,0.000000,0.419295,0.913852,0.000000,0.000000,0.000000,0.000000
1,111,111_000000000001_keypoints.json,1,0.347309,0.214129,0.347309,0.214129,0.302370,0.394282,0.302370,0.496641,0.300065,0.607190,0.325416,0.650181,0.354223,0.678841,0.409533,0.678841,0.479824,0.678841,0.608881,0.406565,0.497108,0.678841,0.536286,0.678841,0.560484,0.619473,0.576617,0.560104,0.596206,0.504830,0.607729,0.443414,0.611186,0.379951,0.388792,0.183421,0.419904,...,0.478671,0.656322,0.453321,0.627661,0.441798,0.592859,0.487890,0.611284,0.505174,0.623567,0.515545,0.623567,0.524763,0.627661,0.515545,0.623567,0.504022,0.623567,0.487890,0.613331,0.440646,0.279639,0.577769,0.341055,0.518980,0.428901,0.282778,0.796686,0.141691,0.796663,0.000000,0.000000,0.000000,0.000000,0.419276,0.916572,0.000000,0.000000,0.000000,0.000000
2,111,111_000000000002_keypoints.json,1,0.304425,0.628224,0.422798,0.283163,0.304425,0.381169,0.304425,0.507759,0.304425,0.609848,0.323962,0.652725,0.351544,0.679268,0.545767,0.609848,0.404410,0.679268,0.608976,0.411795,0.497499,0.679268,0.535424,0.679268,0.557260,0.626182,0.574498,0.566970,0.592886,0.509801,0.606677,0.446506,0.610125,0.381169,0.387171,0.185158,0.415902,...,0.480260,0.658850,0.453827,0.630265,0.440036,0.593513,0.488305,0.613931,0.504394,0.624140,0.517036,0.626182,0.526230,0.630265,0.515887,0.626182,0.503245,0.626182,0.488305,0.618015,0.437738,0.281122,0.575648,0.342375,0.518950,0.445235,0.279717,0.788509,0.150878,0.791206,0.000000,0.000000,0.000000,0.000000,0.417758,0.916585,0.000000,0.000000,0.000000,0.000000
3,111,111_000000000003_keypoints.json,1,0.304477,0.630774,0.422831,0.283726,0.304477,0.381716,0.304477,0.510328,0.304477,0.606276,0.324011,0.649147,0.351588,0.681810,0.553824,0.612401,0.407893,0.681810,0.608980,0.402131,0.498669,0.681810,0.537737,0.667520,0.559570,0.624649,0.575657,0.563406,0.596340,0.506245,0.608980,0.445001,0.611278,0.381716,0.386060,0.185737,0.415936,...,0.480284,0.659354,0.453855,0.630774,0.442365,0.596069,0.489477,0.614442,0.505563,0.626691,0.517054,0.626691,0.525098,0.630774,0.517054,0.626691,0.504414,0.626691,0.489477,0.618525,0.440067,0.283726,0.575657,0.342929,0.522028,0.445276,0.301187,0.799408,0.150882,0.774929,0.000000,0.000000,0.000000,0.000000,0.414691,0.911131,0.000000,0.000000,0.000000,0.000000
4,111,111_000000000004_keypoints.json,1,0.308436,0.638000,0.318619,0.123412,0.308436,0.392767,0.308436,0.515383,0.308436,0.609858,0.326539,0.652071,0.354825,0.676192,0.397819,0.676192,0.408002,0.676192,0.608264,0.398797,0.493990,0.676192,0.540378,0.660111,0.560744,0.621919,0.579978,0.559606,0.598081,0.503323,0.608264,0.443019,0.612790,0.380706,0.386505,0.187735,0.418185,...,0.480413,0.658101,0.454390,0.633980,0.443076,0.601818,0.488333,0.619909,0.504173,0.623929,0.517750,0.625939,0.524539,0.633980,0.516618,0.627949,0.503041,0.625939,0.488333,0.621919,0.438550,0.284221,0.576584,0.344524,0.520474,0.445237,0.000000,0.000000,0.000000,0.00

In [0]:
raw_data.to_csv(CSV_FOLDER + 'cleaned_data.csv')
need_norm_data.to_csv(CSV_FOLDER + 'need_norm_data.csv')

In [0]:
need_norm_data

,vid,frame,label,f_0,f_1,f_2,f_3,f_4,f_5,f_6,f_7,f_8,f_9,f_10,f_11,f_12,f_13,f_14,f_15,f_16,f_17,f_18,f_19,f_20,f_21,f_22,f_23,f_24,f_25,f_26,f_27,f_28,f_29,f_30,f_31,f_32,f_33,f_34,f_35,f_36,...,f_116,f_117,f_118,f_119,f_120,f_121,f_122,f_123,f_124,f_125,f_126,f_127,f_128,f_129,f_130,f_131,f_132,f_133,f_134,f_135,f_136,f_137,f_138,f_139,p_0,p_1,p_2,p_3,p_4,p_5,p_6,p_7,p_8,p_9,p_10,p_11,p_12,p_13,p_14,p_15
11836,43,43_000000000000_keypoints.json,1,515.250,239.873,513.874,281.171,515.250,321.092,524.887,363.767,533.146,405.065,551.042,446.363,585.457,472.518,624.002,495.920,666.676,495.920,710.727,495.920,749.272,480.778,780.934,449.116,801.583,407.818,811.219,365.143,811.219,321.092,808.466,275.664,807.089,232.990,535.899,215.094,548.289,...,636.391,428.467,612.989,420.207,603.353,402.312,636.391,403.688,657.040,405.065,679.066,400.935,714.857,394.052,679.066,402.312,657.040,405.065,636.391,405.065,579.951,238.496,717.610,228.860,648.916,308.137,686.184,639.175,343.344,654.860,0.0,0.0,0.0,0.0,1038.800,650.911,0.0,0.0,0.0,0.0
11837,43,43_000000000001_keypoints.json,1,515.256,240.356,513.879,281.657,515.256,321.581,524.892,364.258,534.529,406.935,552.426,448.236,585.467,473.016,624.014,496.420,665.315,496.420,712.122,496.420,749.293,479.900,780.957,448.236,801.607,406.935,811.244,364.258,811.244,320.204,808.490,276.150,808.490,232.096,535.906,212.822,548.296,...,636.404,430.339,614.377,422.079,603.364,402.805,636.404,404.182,657.055,404.182,679.082,400.052,714.876,394.545,679.082,401.429,657.055,405.559,636.404,405.559,579.960,238.979,717.629,229.342,648.908,308.090,686.191,637.230,345.301,652.880,0.0,0.0,0.0,0.0,1038.820,648.986,0.0,0.0,0.0,0.0
11838,43,43_000000000002_keypoints.json,1,515.259,240.354,513.883,281.652,515.259,322.950,524.896,365.625,534.532,406.924,553.804,448.222,586.843,474.378,624.012,496.403,665.310,496.403,710.738,496.403,747.907,481.261,780.945,449.599,801.594,408.300,811.231,364.249,811.231,320.197,808.478,276.146,808.478,232.094,535.908,212.821,548.298,...,636.401,433.079,612.999,424.820,601.986,402.794,636.401,402.794,657.050,402.794,677.699,395.911,716.245,394.534,679.076,404.171,657.050,406.924,636.401,406.924,579.960,238.977,717.621,229.341,648.912,308.097,688.158,637.215,347.261,648.974,0.0,0.0,0.0,0.0,1038.810,648.988,0.0,0.0,0.0,0.0
11839,43,43_000000000003_keypoints.json,1,515.277,240.365,513.901,283.039,515.277,324.335,524.913,365.632,534.549,405.552,551.068,446.849,584.105,473.004,622.649,496.405,663.945,496.405,710.748,496.405,747.916,479.887,780.953,449.602,801.601,406.929,811.237,364.256,811.237,321.582,808.484,276.156,808.484,232.106,535.925,214.211,548.315,...,636.414,435.837,613.013,426.201,602.000,404.176,636.414,402.799,657.063,402.799,677.711,395.916,717.631,394.540,679.088,406.929,657.063,412.435,636.414,412.435,579.975,238.989,717.631,229.353,648.940,308.117,690.112,635.267,347.264,648.971,0.0,0.0,0.0,0.0,1038.800,648.956,0.0,0.0,0.0,0.0
11840,43,43_000000000004_keypoints.json,1,515.262,240.857,513.886,283.531,516.639,324.828,524.898,367.502,535.911,407.422,553.806,448.720,586.844,474.874,624.011,496.900,666.685,496.900,712.112,496.900,749.280,481.757,782.317,450.096,801.589,408.799,811.225,366.125,811.225,322.075,808.472,276.648,808.472,232.598,535.911,213.326,548.300,...,636.401,437.707,612.999,426.694,601.986,404.669,636.401,403.293,657.049,403.293,679.074,396.410,720.372,395.033,679.074,410.176,657.049,414.305,636.401,414.305,579.961,238.104,717.618,229.844,648.983,308.148,688.158,633.285,345.308,650.937,0.0,0.0,0.0,0.0,1036.890,648.943,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23689,36,36_000000000271_keypoints.json,0,535.523,258.230,542.895,286.491,552.725,315.982,558.869,345.473,5

In [0]:
need_norm_data2 = raw_data.loc[raw_data['vid'].isin([36, 37, 38, 39, 40, 41, 42])]

In [0]:
need_norm_data2.head(50)

,vid,frame,label,f_0,f_1,f_2,f_3,f_4,f_5,f_6,f_7,f_8,f_9,f_10,f_11,f_12,f_13,f_14,f_15,f_16,f_17,f_18,f_19,f_20,f_21,f_22,f_23,f_24,f_25,f_26,f_27,f_28,f_29,f_30,f_31,f_32,f_33,f_34,f_35,f_36,...,f_116,f_117,f_118,f_119,f_120,f_121,f_122,f_123,f_124,f_125,f_126,f_127,f_128,f_129,f_130,f_131,f_132,f_133,f_134,f_135,f_136,f_137,f_138,f_139,p_0,p_1,p_2,p_3,p_4,p_5,p_6,p_7,p_8,p_9,p_10,p_11,p_12,p_13,p_14,p_15
13337,41,41_000000000003_keypoints.json,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.0,0.0,0.0,0.0
13338,41,41_000000000005_keypoints.json,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.0,0.0,0.0,0.0
13339,41,41_000000000006_keypoints.json,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.0,0.0,0.0,0.0
13340,41,41_000000000008_keypoints.json,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.0,0.0,0.0,0.0
13341,41,41_000000000009_keypoints.json,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.0,0.0,0.0,0.0
13342,41,41_000000000010_keypoints.json,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.0,0.0,0.0,0.0
13343,41,41_000000000014_keypoints.json,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.0,0.0,0.0,0.0
13344,41,41_000000000017_keypoints.json,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,826.964,220.234,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.0,0.0,0.0,0.0
13345,41,41_000000000019_keypoints.json,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.000,902.637,218.758,0.0,0.0,0.0,0.0,0.0,0.0,946.274,234.762,0.0,0.0,0.0,0.0
13346,41,41_000000000020_keypoints.json,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.

In [0]:
need_norm_data

,vid,frame,label,f_0,f_1,f_2,f_3,f_4,f_5,f_6,f_7,f_8,f_9,f_10,f_11,f_12,f_13,f_14,f_15,f_16,f_17,f_18,f_19,f_20,f_21,f_22,f_23,f_24,f_25,f_26,f_27,f_28,f_29,f_30,f_31,f_32,f_33,f_34,f_35,f_36,...,f_116,f_117,f_118,f_119,f_120,f_121,f_122,f_123,f_124,f_125,f_126,f_127,f_128,f_129,f_130,f_131,f_132,f_133,f_134,f_135,f_136,f_137,f_138,f_139,p_0,p_1,p_2,p_3,p_4,p_5,p_6,p_7,p_8,p_9,p_10,p_11,p_12,p_13,p_14,p_15
11836,43,43_000000000000_keypoints.json,1,515.250,239.873,513.874,281.171,515.250,321.092,524.887,363.767,533.146,405.065,551.042,446.363,585.457,472.518,624.002,495.920,666.676,495.920,710.727,495.920,749.272,480.778,780.934,449.116,801.583,407.818,811.219,365.143,811.219,321.092,808.466,275.664,807.089,232.990,535.899,215.094,548.289,...,636.391,428.467,612.989,420.207,603.353,402.312,636.391,403.688,657.040,405.065,679.066,400.935,714.857,394.052,679.066,402.312,657.040,405.065,636.391,405.065,579.951,238.496,717.610,228.860,648.916,308.137,686.184,639.175,343.344,654.860,0.0,0.0,0.0,0.0,1038.800,650.911,0.0,0.0,0.0,0.0
11837,43,43_000000000001_keypoints.json,1,515.256,240.356,513.879,281.657,515.256,321.581,524.892,364.258,534.529,406.935,552.426,448.236,585.467,473.016,624.014,496.420,665.315,496.420,712.122,496.420,749.293,479.900,780.957,448.236,801.607,406.935,811.244,364.258,811.244,320.204,808.490,276.150,808.490,232.096,535.906,212.822,548.296,...,636.404,430.339,614.377,422.079,603.364,402.805,636.404,404.182,657.055,404.182,679.082,400.052,714.876,394.545,679.082,401.429,657.055,405.559,636.404,405.559,579.960,238.979,717.629,229.342,648.908,308.090,686.191,637.230,345.301,652.880,0.0,0.0,0.0,0.0,1038.820,648.986,0.0,0.0,0.0,0.0
11838,43,43_000000000002_keypoints.json,1,515.259,240.354,513.883,281.652,515.259,322.950,524.896,365.625,534.532,406.924,553.804,448.222,586.843,474.378,624.012,496.403,665.310,496.403,710.738,496.403,747.907,481.261,780.945,449.599,801.594,408.300,811.231,364.249,811.231,320.197,808.478,276.146,808.478,232.094,535.908,212.821,548.298,...,636.401,433.079,612.999,424.820,601.986,402.794,636.401,402.794,657.050,402.794,677.699,395.911,716.245,394.534,679.076,404.171,657.050,406.924,636.401,406.924,579.960,238.977,717.621,229.341,648.912,308.097,688.158,637.215,347.261,648.974,0.0,0.0,0.0,0.0,1038.810,648.988,0.0,0.0,0.0,0.0
11839,43,43_000000000003_keypoints.json,1,515.277,240.365,513.901,283.039,515.277,324.335,524.913,365.632,534.549,405.552,551.068,446.849,584.105,473.004,622.649,496.405,663.945,496.405,710.748,496.405,747.916,479.887,780.953,449.602,801.601,406.929,811.237,364.256,811.237,321.582,808.484,276.156,808.484,232.106,535.925,214.211,548.315,...,636.414,435.837,613.013,426.201,602.000,404.176,636.414,402.799,657.063,402.799,677.711,395.916,717.631,394.540,679.088,406.929,657.063,412.435,636.414,412.435,579.975,238.989,717.631,229.353,648.940,308.117,690.112,635.267,347.264,648.971,0.0,0.0,0.0,0.0,1038.800,648.956,0.0,0.0,0.0,0.0
11840,43,43_000000000004_keypoints.json,1,515.262,240.857,513.886,283.531,516.639,324.828,524.898,367.502,535.911,407.422,553.806,448.720,586.844,474.874,624.011,496.900,666.685,496.900,712.112,496.900,749.280,481.757,782.317,450.096,801.589,408.799,811.225,366.125,811.225,322.075,808.472,276.648,808.472,232.598,535.911,213.326,548.300,...,636.401,437.707,612.999,426.694,601.986,404.669,636.401,403.293,657.049,403.293,679.074,396.410,720.372,395.033,679.074,410.176,657.049,414.305,636.401,414.305,579.961,238.104,717.618,229.844,648.983,308.148,688.158,633.285,345.308,650.937,0.0,0.0,0.0,0.0,1036.890,648.943,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23689,36,36_000000000271_keypoints.json,0,535.523,258.230,542.895,286.491,552.725,315.982,558.869,345.473,5

In [0]:
raw_data.loc[raw_data['vid'] == '41']

,vid,frame,label,f_0,f_1,f_2,f_3,f_4,f_5,f_6,f_7,f_8,f_9,f_10,f_11,f_12,f_13,f_14,f_15,f_16,f_17,f_18,f_19,f_20,f_21,f_22,f_23,f_24,f_25,f_26,f_27,f_28,f_29,f_30,f_31,f_32,f_33,f_34,f_35,f_36,...,f_116,f_117,f_118,f_119,f_120,f_121,f_122,f_123,f_124,f_125,f_126,f_127,f_128,f_129,f_130,f_131,f_132,f_133,f_134,f_135,f_136,f_137,f_138,f_139,p_0,p_1,p_2,p_3,p_4,p_5,p_6,p_7,p_8,p_9,p_10,p_11,p_12,p_13,p_14,p_15
13337,41,41_000000000003_keypoints.json,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.000,0.000,0.0,0.0
13338,41,41_000000000005_keypoints.json,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.000,0.000,0.0,0.0
13339,41,41_000000000006_keypoints.json,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.000,0.000,0.0,0.0
13340,41,41_000000000008_keypoints.json,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.000,0.000,0.0,0.0
13341,41,41_000000000009_keypoints.json,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.000,0.000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13475,41,41_000000000230_keypoints.json,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,645.062,167.834,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.000,0.000,0.0,0.0
13476,41,41_000000000231_keypoints.json,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,637.804,179.466,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.000,0.000,0.0,0.0
13477,41,41_000000000234_keypoints.json,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,739.654,391.912,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.000,0.000,0.0,0.0
13478,41,41_000000000236_keypoints.json,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

In [0]:
need_norm_data['vid'] = need_norm_data['vid'].astype(int)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [0]:
def normalization(dataset, x, y):
  for i in range(0, 140, 2):
    x_name = 'f_' + str(i)
    y_name = 'f_' + str(i + 1)
    dataset[x_name] = dataset[x_name] / x
    dataset[y_name] = dataset[y_name] / y
  for i in range(0, 16, 2):
    x_name = 'p_' + str(i)
    y_name = 'p_' + str(i + 1)
    dataset[x_name] = dataset[x_name] / x
    dataset[y_name] = dataset[y_name] / y

In [0]:
def norm_data(vid, raw_data, x, y):
  data = raw_data.loc[raw_data['vid'] == vid]
  normalization(data, x, y)
  raw_data = raw_data.append(data)

In [0]:
data = need_norm_data.loc[need_norm_data['vid'] == 41]
data.drop(data.index[:7], inplace=True)
normalization(data, 1280, 534)

In [0]:
data_1 = need_norm_data.loc[need_norm_data['vid'].isin([31, 33, 36, 37, 38, 39, 40, 42, 43])]
normalization(data_1, 1280, 720)

In [0]:
data = data.append(data_1)

In [0]:
sorted(need_norm_data.vid.unique())

[28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 43]

In [0]:
data_2 = need_norm_data.loc[need_norm_data['vid'].isin([28, 29])]
normalization(data_2, 540, 360)
data = data.append(data_2)

In [0]:
data_3 = need_norm_data.loc[need_norm_data['vid'] == 30]
normalization(data_3, 640, 312)
data = data.append(data_3)

In [0]:
data_4 = need_norm_data.loc[need_norm_data['vid'] == 32]
normalization(data_4, 286, 360)
data = data.append(data_4)

In [0]:
data_5 = need_norm_data.loc[need_norm_data['vid'] == 34]
normalization(data_5, 640, 358)
data = data.append(data_5)

In [0]:
data_6 = need_norm_data.loc[need_norm_data['vid'] == 35]
normalization(data_6, 406, 720)
data = data.append(data_6)

In [0]:
data.vid.unique()

array([41, 43, 40, 38, 33, 31, 39, 37, 36, 29, 28, 30, 32, 34, 35])

In [0]:
raw_data = raw_data.append(data)

In [0]:
raw_data.to_csv(CSV_FOLDER + 'cleaned_data_all_pos_neg.csv')

In [0]:
import json
import pandas as pd
import numpy as np
import os, os.path
import time
import csv

In [0]:
ls './drive/My Drive/636/data/part6_csv/'

 cleaned_data_all_pos_neg.csv   half.csv          half_n.csv           pos.csv
 cleaned_data.csv              'half_n (1).csv'   need_norm_data.csv


In [0]:
CSV_FOLDER = './drive/My Drive/636/data/part6_csv/'
raw_data_h = pd.read_csv(CSV_FOLDER + 'half.csv')
raw_data_h.drop(raw_data_h.index[raw_data_h['vid'] == 'vid'], inplace = True)
raw_data_h.reset_index(drop=True)
raw_data_h.vid.unique()

/usr/local/lib/python3.6/dist-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


array([101])

In [0]:
CSV_FOLDER = './drive/My Drive/636/data/part6_csv/'
raw_data_0 = pd.read_csv(CSV_FOLDER + 'half_n.csv')
raw_data_0.drop(raw_data_0.index[raw_data_0['vid'] == 'vid'], inplace = True)
raw_data_0.reset_index(drop=True)

,vid,frame,label,f_0,f_1,f_2,f_3,f_4,f_5,f_6,f_7,f_8,f_9,f_10,f_11,f_12,f_13,f_14,f_15,f_16,f_17,f_18,f_19,f_20,f_21,f_22,f_23,f_24,f_25,f_26,f_27,f_28,f_29,f_30,f_31,f_32,f_33,f_34,f_35,f_36,...,f_116,f_117,f_118,f_119,f_120,f_121,f_122,f_123,f_124,f_125,f_126,f_127,f_128,f_129,f_130,f_131,f_132,f_133,f_134,f_135,f_136,f_137,f_138,f_139,p_0,p_1,p_2,p_3,p_4,p_5,p_6,p_7,p_8,p_9,p_10,p_11,p_12,p_13,p_14,p_15
0,42,42_000000000000_keypoints.json,0,520.686,212.742,515.056,257.786,519.279,301.422,524.909,346.466,533.355,390.102,547.431,426.7,578.399,459.075,614.997,484.412,655.818,484.412,699.454,484.412,738.867,460.483,771.242,433.738,786.726,398.548,796.579,357.727,800.802,315.498,806.433,274.677,809.248,232.449,544.616,176.144,561.507,...,645.964,409.809,623.443,399.955,613.589,377.433,648.78,367.58,662.856,367.58,681.155,367.58,710.715,380.249,681.155,388.694,662.856,388.694,648.78,385.879,602.328,208.519,737.46,212.742,666.631,276.747,647.011,643.091,337.475,603.909,0.0,0.0,0.0,0.0,1011.4,637.217,0.0,0.0,0.0,0.0
1,42,42_000000000001_keypoints.json,0,520.694,209.945,515.101,253.293,516.499,298.041,523.491,342.788,527.686,386.137,547.263,425.29,573.832,456.054,610.189,479.826,649.342,479.826,695.488,479.826,736.04,464.444,768.202,432.282,784.982,397.323,794.771,353.975,798.966,316.219,804.559,272.87,808.754,230.92,543.068,173.587,561.246,...,639.554,409.909,617.18,397.323,605.994,374.95,646.546,363.763,660.529,365.161,680.106,365.161,712.268,377.747,680.106,388.933,660.529,388.933,643.749,387.535,601.799,207.148,736.04,212.741,666.59,274.839,658.783,633.291,339.438,605.882,0.0,0.0,0.0,0.0,1011.41,637.209,0.0,0.0,0.0,0.0
2,42,42_000000000002_keypoints.json,0,518.196,208.531,515.414,253.042,515.414,296.163,523.76,340.674,527.933,385.185,547.406,426.914,573.835,456.124,610.0,476.989,651.729,476.989,694.849,476.989,735.188,461.688,767.18,429.696,785.263,394.922,793.609,353.192,800.563,315.636,804.736,272.516,807.518,229.396,543.233,172.366,559.925,...,637.82,408.831,615.564,396.313,603.045,372.666,641.992,362.929,660.075,362.929,678.158,362.929,712.932,376.839,678.158,392.14,658.684,392.14,640.602,386.576,600.263,205.749,735.188,211.313,666.564,274.785,656.804,637.215,339.447,605.876,0.0,0.0,0.0,0.0,1009.45,635.232,0.0,0.0,0.0,0.0
3,42,42_000000000003_keypoints.json,0,515.739,208.081,514.352,251.086,515.739,295.479,522.676,339.871,526.838,382.877,546.259,424.495,572.617,453.627,610.073,475.824,653.079,475.824,694.697,475.824,734.927,460.564,766.835,428.657,784.869,393.975,793.193,352.357,798.742,314.901,802.904,271.895,807.065,228.89,542.097,172.012,560.132,...,636.432,410.622,614.235,395.362,601.75,371.779,641.981,362.068,658.628,362.068,678.05,362.068,712.731,374.553,678.05,392.588,658.628,392.588,640.593,387.039,598.975,205.307,733.54,209.468,664.632,274.772,647.016,637.214,341.381,609.789,0.0,0.0,0.0,0.0,1009.45,631.344,0.0,0.0,0.0,0.0
4,42,42_000000000004_keypoints.json,0,515.453,209.66,514.086,253.408,515.453,297.156,522.289,340.904,529.125,383.285,546.897,425.667,578.341,457.11,612.52,472.149,654.901,472.149,698.649,472.149,735.561,458.478,767.005,428.401,786.145,392.855,795.715,353.209,798.449,314.929,803.918,271.181,808.019,228.8,540.062,172.748,557.834,...,641.229,413.362,613.887,398.324,601.583,370.981,643.964,360.044,659.002,362.779,676.775,361.411,710.953,376.45,676.775,391.488,659.002,391.488,643.964,390.121,598.848,205.559,731.46,209.66,664.659,274.768,656.802,643.094,345.307,613.701,0.0,0.0,0.0,0.0,1009.46,631.34,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1512,45,45_000000000012_keypoints.json,0,517.191,252.079,511.855,280.894,511.855,308.641,512.922,336.388,519.325,366.27,532.131,391.883,551.341,413.227,571.618,435.638,59

In [0]:
raw_data_0.vid.unique()

array(['42', '45'], dtype=object)

In [0]:
CSV_FOLDER = './drive/My Drive/636/data/part6_csv/'
raw_data_1 = pd.read_csv(CSV_FOLDER + 'half_n (1).csv')
raw_data_1.drop(raw_data_1.index[raw_data_1['vid'] == 'vid'], inplace = True)
raw_data_1.reset_index(drop=True)

,vid,frame,label,f_0,f_1,f_2,f_3,f_4,f_5,f_6,f_7,f_8,f_9,f_10,f_11,f_12,f_13,f_14,f_15,f_16,f_17,f_18,f_19,f_20,f_21,f_22,f_23,f_24,f_25,f_26,f_27,f_28,f_29,f_30,f_31,f_32,f_33,f_34,f_35,f_36,...,f_116,f_117,f_118,f_119,f_120,f_121,f_122,f_123,f_124,f_125,f_126,f_127,f_128,f_129,f_130,f_131,f_132,f_133,f_134,f_135,f_136,f_137,f_138,f_139,p_0,p_1,p_2,p_3,p_4,p_5,p_6,p_7,p_8,p_9,p_10,p_11,p_12,p_13,p_14,p_15
0,101,101_000000000000_keypoints.json,1,0.294216,0.403116,0.297334,0.469606,0.302532,0.530554,0.311888,0.584115,0.330601,0.632135,0.352432,0.672767,0.377381,0.706012,0.40545,0.722634,0.436637,0.724481,0.461587,0.709705,0.482378,0.680155,0.495893,0.643216,0.505249,0.600737,0.511486,0.554564,0.515644,0.504697,0.518763,0.456677,0.517724,0.404963,0.336838,0.386494,0.352432,...,0.42832,0.635829,0.415846,0.633982,0.408569,0.619206,0.4304,0.606278,0.442874,0.606278,0.452231,0.604431,0.466785,0.608125,0.452231,0.617359,0.442874,0.619206,0.42936,0.619206,0.385698,0.427126,0.47718,0.414198,0.440743,0.510499,0.402422,0.889324,0.203029,0.9193,0.0,0.0,0.0,0.0,0.629401,0.946538,0.0,0.0,0.0,0.0
1,101,101_000000000001_keypoints.json,1,0.294216,0.403109,0.297334,0.469599,0.302532,0.530548,0.311889,0.584109,0.330601,0.632129,0.352432,0.672762,0.376343,0.706007,0.404411,0.724476,0.436638,0.724476,0.461588,0.709701,0.48134,0.68015,0.495895,0.643211,0.505251,0.600731,0.511488,0.554558,0.515647,0.502844,0.518765,0.45667,0.517726,0.404956,0.336839,0.386486,0.352432,...,0.428322,0.635823,0.415847,0.633976,0.40857,0.619201,0.430401,0.606272,0.442876,0.606272,0.452232,0.604425,0.466786,0.608119,0.452232,0.615507,0.442876,0.619201,0.429361,0.619201,0.385699,0.427119,0.477182,0.41419,0.440745,0.510499,0.40243,0.883886,0.201513,0.922037,0.0,0.0,0.0,0.0,0.629408,0.946522,0.0,0.0,0.0,0.0
2,101,101_000000000002_keypoints.json,1,0.293373,0.408681,0.297547,0.473568,0.302765,0.532894,0.312156,0.584804,0.331983,0.631151,0.351809,0.670084,0.376854,0.703454,0.405028,0.725701,0.436334,0.725701,0.461378,0.712724,0.481205,0.681207,0.495814,0.644129,0.505205,0.601489,0.511466,0.555141,0.51564,0.505085,0.518771,0.458737,0.517727,0.406827,0.336157,0.390142,0.352853,...,0.427986,0.638567,0.415463,0.636713,0.409202,0.620028,0.429029,0.608904,0.442595,0.608904,0.451986,0.607051,0.466595,0.608904,0.451986,0.61632,0.442595,0.621882,0.430073,0.620028,0.385202,0.429074,0.477031,0.416097,0.440762,0.510648,0.403939,0.886591,0.2015,0.919306,0.0,0.0,0.0,0.0,0.630927,0.946575,0.0,0.0,0.0,0.0
3,101,101_000000000003_keypoints.json,1,0.29337,0.410549,0.2965,0.473585,0.302761,0.532912,0.312153,0.584824,0.331981,0.631174,0.351808,0.671962,0.376854,0.703479,0.40503,0.725727,0.436336,0.725727,0.461381,0.712749,0.481209,0.683086,0.495818,0.644152,0.50521,0.603364,0.511472,0.557014,0.515646,0.505102,0.518777,0.458753,0.517733,0.406841,0.336155,0.390155,0.352852,...,0.427988,0.63859,0.415465,0.636736,0.409204,0.62005,0.430075,0.608926,0.442597,0.608926,0.451989,0.607072,0.466599,0.608926,0.451989,0.618196,0.442597,0.621904,0.430075,0.62005,0.385202,0.429089,0.477035,0.416111,0.440766,0.510643,0.40241,0.88389,0.203048,0.922034,0.0,0.0,0.0,0.0,0.630918,0.94657,0.0,0.0,0.0,0.0
4,101,101_000000000004_keypoints.json,1,0.294831,0.405281,0.298994,0.471851,0.304199,0.531025,0.312525,0.582802,0.33022,0.629031,0.352077,0.669713,0.377057,0.701149,0.404119,0.725188,0.436385,0.727037,0.461365,0.712244,0.480101,0.682657,0.495713,0.643824,0.505081,0.601293,0.512367,0.555064,0.515489,0.506985,0.519652,0.460756,0.517571,0.40713,0.337505,0.388638,0.354159,...,0.429099,0.640126,0.415568,0.636428,0.410364,0.619785,0.43014,0.610539,0.44263,0.60869,0.451998,0.60869,0.46657,0.610539,0.453039,0.614237,0.443671,0.619785,0.43014,0.619785,0.386425,0.431169,0.478019,0.418225,0.440754,0.510637,0.402426,0.883897,0.201512,0.922027,0.0,0.0,0.0,0.0,0.630925,0.949246,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,..

In [0]:
raw_data_1.vid.unique()

array(['101', '93', '91'], dtype=object)

In [0]:
raw_data = raw_data_1.append(raw_data_0)

In [0]:
raw_data['label'] = raw_data['label'].astype(int)
raw_data['vid'] = raw_data['vid'].astype(int)
cols = list(raw_data.columns)
cols.remove('vid')
cols.remove('frame')
cols.remove('label')
for col in cols:
    raw_data[col] = raw_data[col].astype(float)

In [0]:
raw_data.to_csv(CSV_FOLDER + 'cleaned_data_half.csv')

In [0]:
ls './drive/My Drive/636/dataset/part4'

dataset.csv  dataset_dropped.csv  neg.csv  pos.csv


In [0]:
PART4_FOLDER = './drive/My Drive/636/dataset/part4/'
raw_data = pd.read_csv(PART4_FOLDER + 'dataset.csv')

In [0]:
raw_data

,person_id,p_0,p_1,p_2,p_3,p_4,p_5,p_6,p_7,p_8,p_9,p_10,p_11,p_12,p_13,p_14,p_15,p_16,p_17,p_18,p_19,p_20,p_21,p_22,p_23,p_24,p_25,p_26,p_27,p_28,p_29,p_30,p_31,p_32,p_33,p_34,p_35,p_36,p_37,p_38,...,rh_24,rh_25,rh_26,rh_27,rh_28,rh_29,rh_30,rh_31,rh_32,rh_33,rh_34,rh_35,rh_36,rh_37,rh_38,rh_39,rh_40,rh_41,rh_42,rh_43,rh_44,rh_45,rh_46,rh_47,rh_48,rh_49,rh_50,rh_51,rh_52,rh_53,rh_54,rh_55,rh_56,rh_57,rh_58,rh_59,rh_60,rh_61,rh_62,label
0,1,948.234,571.425,0.802683,954.130,1004.180,0.337380,571.496,1015.930,0.189887,0.0,0.0,0.0,0.0,0.0,0.0,1383.820,1021.810,0.217755,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
1,1,945.317,565.614,0.780813,957.088,1010.030,0.325065,559.714,1033.570,0.182766,0.0,0.0,0.0,0.0,0.0,0.0,1374.990,1015.920,0.214545,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
2,1,945.258,556.862,0.777462,951.220,1021.750,0.329812,553.829,1039.470,0.183845,0.0,0.0,0.0,0.0,0.0,0.0,1375.020,1007.080,0.226302,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
3,1,951.020,553.964,0.825141,954.132,1018.900,0.341418,550.851,1045.360,0.171063,0.0,0.0,0.0,0.0,0.0,0.0,1375.030,1007.100,0.257979,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
4,1,951.204,553.965,0.842606,951.132,1013.010,0.355129,559.727,1042.410,0.183300,0.0,0.0,0.0,0.0,0.0,0.0,1369.110,995.317,0.238134,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6776,165,654.863,274.787,0.802982,619.646,509.894,0.535690,398.166,509.836,0.348031,0.0,0.0,0.0,0.0,0.0,0.0,839.004,517.725,0.414678,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1
6777,165,652.930,274.881,0.812548,621.518,511.850,0.530628,398.168,509.909,0.332508,0.0,0.0,0.0,0.0,0.0,0.0,839.014,519.662,0.419507,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1
6778,165,651.008,274.848,0.826359,619.579,509.925,0.537220,394.306,509.903,0.335926,0.0,0.0,0.0,0.0,0.0,0.0,838.982,519.653,0.425953,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1
6779,165,650.993,274.795,0.835295,617.685,511.866,0.542406,396.236,509.909,0.346731,0.0,0.0,0.0,0.0,0.0,0.0,837.079,519.682,0.417218,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.

In [0]:
N_FACE = 70
N_POSE = 8
COL_NAMES = ['vid', 'frame', 'label']
COL_NAMES += ['f_' + str(i) for i in range(N_FACE * 2)]
COL_NAMES += ['p_' + str(i) for i in range(N_POSE * 2)]

In [0]:
raw_data.person_id = raw_data.person_id + 154

In [0]:
cleaned_data = pd.DataFrame(columns=COL_NAMES)
cleaned_data['vid'] = raw_data['person_id']
cleaned_data['label'] = raw_data['label']
cleaned_data.label[cleaned_data.label == -1] = 0

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [0]:
for i in range(70):
  cleaned_data['f_' + str(i * 2)] = raw_data['f_' + str(i * 3)]
  cleaned_data['f_' + str(i * 2 + 1)] = raw_data['f_' + str(i * 3 + 1)]
for i in range(8):
  cleaned_data['p_' + str(i * 2)] = raw_data['p_' + str(i * 3)]
  cleaned_data['p_' + str(i * 2 + 1)] = raw_data['p_' + str(i * 3 + 1)]

In [0]:
cleaned_data.vid = cleaned_data.vid - 154 * 3

In [0]:
cleaned_data

,vid,frame,label,f_0,f_1,f_2,f_3,f_4,f_5,f_6,f_7,f_8,f_9,f_10,f_11,f_12,f_13,f_14,f_15,f_16,f_17,f_18,f_19,f_20,f_21,f_22,f_23,f_24,f_25,f_26,f_27,f_28,f_29,f_30,f_31,f_32,f_33,f_34,f_35,f_36,...,f_116,f_117,f_118,f_119,f_120,f_121,f_122,f_123,f_124,f_125,f_126,f_127,f_128,f_129,f_130,f_131,f_132,f_133,f_134,f_135,f_136,f_137,f_138,f_139,p_0,p_1,p_2,p_3,p_4,p_5,p_6,p_7,p_8,p_9,p_10,p_11,p_12,p_13,p_14,p_15
0,155,NaN,1,763.665,423.757,769.403,477.315,778.967,530.872,790.443,580.605,815.310,628.424,834.437,672.418,865.042,714.499,907.123,745.103,954.942,754.667,1000.850,745.103,1033.370,712.586,1062.060,670.505,1086.920,628.424,1107.960,580.605,1113.700,532.785,1123.270,484.966,1125.180,427.583,801.920,408.455,819.135,...,924.338,674.331,897.559,657.116,891.820,626.511,924.338,634.162,941.553,636.075,960.680,634.162,1002.760,628.424,962.593,655.203,941.553,655.203,924.338,653.290,865.042,456.274,1029.540,467.751,948.234,571.425,954.130,1004.180,571.496,1015.930,0.0,0.0,0.0,0.0,1383.820,1021.810,0.0,0.0,0.0,0.0
1,155,NaN,1,764.497,420.297,766.441,476.678,778.106,531.114,787.827,579.718,811.157,626.378,830.598,671.094,861.705,713.865,906.421,743.028,953.080,748.860,999.740,739.139,1030.850,708.033,1060.010,669.150,1085.280,624.434,1106.670,577.774,1110.560,531.114,1120.280,482.510,1126.110,422.242,797.548,395.023,815.045,...,923.918,671.094,896.700,653.596,890.867,624.434,923.918,626.378,939.471,632.211,960.857,626.378,1001.680,624.434,962.801,653.596,941.415,653.596,923.918,651.652,863.649,453.348,1026.960,465.013,945.317,565.614,957.088,1010.030,559.714,1033.570,0.0,0.0,0.0,0.0,1374.990,1015.920,0.0,0.0,0.0,0.0
2,155,NaN,1,760.839,422.602,762.838,478.585,774.835,528.570,790.830,578.555,808.824,626.540,832.817,670.527,864.807,706.516,904.795,736.507,952.781,748.503,998.767,736.507,1030.760,704.517,1058.750,668.528,1082.740,624.541,1100.740,576.556,1110.730,528.570,1114.730,480.585,1122.730,426.601,796.828,400.609,818.821,...,920.790,670.527,898.797,654.532,886.801,622.542,920.790,624.541,940.784,626.540,964.777,624.541,1000.770,622.542,964.777,642.536,942.784,648.534,920.790,642.536,858.809,450.594,1022.760,460.591,945.258,556.862,951.220,1021.750,553.829,1039.470,0.0,0.0,0.0,0.0,1375.020,1007.080,0.0,0.0,0.0,0.0
3,155,NaN,1,760.238,424.103,764.239,476.111,776.241,528.118,792.243,580.125,810.245,626.131,838.249,670.137,870.253,706.142,906.258,734.146,954.265,748.148,1002.270,734.146,1034.280,702.141,1064.280,664.136,1084.280,620.130,1102.290,574.124,1112.290,526.117,1116.290,476.111,1120.290,422.103,796.243,398.100,818.246,...,926.261,668.137,902.258,652.135,890.256,620.130,926.261,622.131,948.264,626.131,968.267,622.131,1004.270,620.130,970.267,640.133,950.264,646.134,926.261,640.133,860.252,452.107,1024.270,456.108,951.020,553.964,954.132,1018.900,550.851,1045.360,0.0,0.0,0.0,0.0,1375.030,1007.100,0.0,0.0,0.0,0.0
4,155,NaN,1,759.816,423.623,763.816,477.617,775.815,527.611,791.813,579.605,809.811,625.599,839.807,669.594,871.803,709.590,911.799,735.587,961.793,747.585,1003.790,733.587,1037.780,701.591,1063.780,663.595,1087.780,619.600,1105.780,573.606,1113.770,525.611,1117.770,477.617,1123.770,425.623,807.811,423.623,821.809,...,933.796,667.595,903.800,651.596,891.801,619.600,933.796,623.600,951.794,623.600,969.792,621.600,1011.790,619.600,971.792,639.598,951.794,647.597,933.796,641.598,863.804,451.620,1029.780,455.619,951.204,553.965,951.132,1013.010,559.727,1042.410,0.0,0.0,0.0,0.0,1369.110,995.317,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6776,319,NaN,0,543.977,231.061,542.875,260.820,542.875,292.783,543.977,323.644,553.897,352.301,572.634,374.345,596.882,393.082,623.335,407.410,650.889,408.513,675.137,399.695,693.874,380.958,710.

In [0]:
def normalization(dataset, x, y):
  for i in range(0, 140, 2):
    x_name = 'f_' + str(i)
    y_name = 'f_' + str(i + 1)
    dataset[x_name] = dataset[x_name] / x
    dataset[y_name] = dataset[y_name] / y
  for i in range(0, 16, 2):
    x_name = 'p_' + str(i)
    y_name = 'p_' + str(i + 1)
    dataset[x_name] = dataset[x_name] / x
    dataset[y_name] = dataset[y_name] / y

In [0]:
normalization(cleaned_data, 1920, 1080)

In [0]:
cleaned_data

,vid,frame,label,f_0,f_1,f_2,f_3,f_4,f_5,f_6,f_7,f_8,f_9,f_10,f_11,f_12,f_13,f_14,f_15,f_16,f_17,f_18,f_19,f_20,f_21,f_22,f_23,f_24,f_25,f_26,f_27,f_28,f_29,f_30,f_31,f_32,f_33,f_34,f_35,f_36,...,f_116,f_117,f_118,f_119,f_120,f_121,f_122,f_123,f_124,f_125,f_126,f_127,f_128,f_129,f_130,f_131,f_132,f_133,f_134,f_135,f_136,f_137,f_138,f_139,p_0,p_1,p_2,p_3,p_4,p_5,p_6,p_7,p_8,p_9,p_10,p_11,p_12,p_13,p_14,p_15
0,155,NaN,1,0.397742,0.392368,0.400731,0.441958,0.405712,0.491548,0.411689,0.537597,0.424641,0.581874,0.434603,0.622609,0.450543,0.661573,0.472460,0.689910,0.497366,0.698766,0.521276,0.689910,0.538214,0.659802,0.553156,0.620838,0.566104,0.581874,0.577063,0.537597,0.580052,0.493319,0.585036,0.449043,0.586031,0.395910,0.417667,0.378199,0.426633,...,0.481426,0.624381,0.467479,0.608441,0.464490,0.580103,0.481426,0.587187,0.490392,0.588958,0.500354,0.587187,0.522271,0.581874,0.501351,0.606669,0.490392,0.606669,0.481426,0.604898,0.450543,0.422476,0.536219,0.433103,0.493872,0.529097,0.496943,0.929796,0.297654,0.940676,0.0,0.0,0.0,0.0,0.720740,0.946120,0.0,0.0,0.0,0.0
1,155,NaN,1,0.398176,0.389164,0.399188,0.441369,0.405264,0.491772,0.410327,0.536776,0.422478,0.579980,0.432603,0.621383,0.448805,0.660986,0.472094,0.687989,0.496396,0.693389,0.520698,0.684388,0.536901,0.655586,0.552089,0.619583,0.565250,0.578180,0.576391,0.534976,0.578417,0.491772,0.583479,0.446769,0.586516,0.390965,0.415390,0.365762,0.424503,...,0.481207,0.621383,0.467031,0.605181,0.463993,0.578180,0.481207,0.579980,0.489308,0.585381,0.500446,0.579980,0.521708,0.578180,0.501459,0.605181,0.490320,0.605181,0.481207,0.603381,0.449817,0.419767,0.534875,0.430568,0.492353,0.523717,0.498483,0.935213,0.291518,0.957009,0.0,0.0,0.0,0.0,0.716141,0.940667,0.0,0.0,0.0,0.0
2,155,NaN,1,0.396270,0.391298,0.397311,0.443134,0.403560,0.489417,0.411891,0.535699,0.421262,0.580130,0.433759,0.620858,0.450420,0.654181,0.471247,0.681951,0.496240,0.693058,0.520191,0.681951,0.536854,0.652331,0.551432,0.619007,0.563927,0.578279,0.573302,0.533848,0.578505,0.489417,0.580589,0.444986,0.584755,0.395001,0.415015,0.370934,0.426469,...,0.479578,0.620858,0.468123,0.606048,0.461876,0.576428,0.479578,0.578279,0.489992,0.580130,0.502488,0.578279,0.521234,0.576428,0.502488,0.594941,0.491033,0.600494,0.479578,0.594941,0.447296,0.417217,0.532687,0.426473,0.492322,0.515613,0.495427,0.946065,0.288453,0.962472,0.0,0.0,0.0,0.0,0.716156,0.932481,0.0,0.0,0.0,0.0
3,155,NaN,1,0.395957,0.392688,0.398041,0.440844,0.404292,0.488998,0.412627,0.537153,0.422003,0.579751,0.436588,0.620497,0.453257,0.653835,0.472009,0.679765,0.497013,0.692730,0.522016,0.679765,0.538687,0.650131,0.554312,0.614941,0.564729,0.574194,0.574109,0.531596,0.579318,0.487145,0.581401,0.440844,0.583484,0.390836,0.414710,0.368611,0.426170,...,0.482428,0.618645,0.469926,0.603829,0.463675,0.574194,0.482428,0.576047,0.493887,0.579751,0.504306,0.576047,0.523057,0.574194,0.505347,0.592716,0.494929,0.598272,0.482428,0.592716,0.448048,0.418618,0.533474,0.422322,0.495323,0.512930,0.496944,0.943426,0.286902,0.967926,0.0,0.0,0.0,0.0,0.716161,0.932500,0.0,0.0,0.0,0.0
4,155,NaN,1,0.395738,0.392244,0.397821,0.442238,0.404070,0.488529,0.412403,0.536671,0.421777,0.579258,0.437399,0.619994,0.454064,0.657028,0.474895,0.681099,0.500934,0.692208,0.522807,0.679247,0.540510,0.649621,0.554052,0.614440,0.566552,0.573704,0.575927,0.531117,0.580089,0.486677,0.582172,0.442238,0.585297,0.394095,0.420735,0.392244,0.428026,...,0.486352,0.618144,0.470729,0.603330,0.464480,0.573704,0.486352,0.577407,0.495726,0.577407,0.505100,0.575556,0.526974,0.573704,0.506142,0.592220,0.495726,0.599627,0.486352,0.594072,0.449898,0.418167,0.536344,0.421869,0.495419,0.512931,0.495381,0.937972,0.291524,0.965194,0.0,0.0,0.0,0.0,0.713078,0.921590,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

In [0]:
rm './drive/My Drive/636/data/part6_csv/part4.csv'

In [0]:
ls ./drive/My\ Drive/636/data/part6_csv/

 cleaned_data_all_pos_neg.csv   half.csv          need_norm_data.csv
 cleaned_data.csv              'half_n (1).csv'   pos.csv
 cleaned_data_half.csv          half_n.csv


In [0]:
cleaned_data.to_csv(CSV_FOLDER + 'part4.csv')

In [0]:
dataset_1 = pd.read_csv(CSV_FOLDER + 'cleaned_data_half.csv', index_col=0)

In [0]:
dataset_0 = pd.read_csv(CSV_FOLDER + 'cleaned_data_all_pos_neg.csv', index_col=0)

In [0]:
dataset = cleaned_data.append(dataset_1, ignore_index=True)
dataset = dataset.append(dataset_0, ignore_index=True)

In [0]:
dataset_3 = pd.read_csv('./drive/My Drive/636/dataset/part5/dataset.csv', index_col=0)

In [0]:
dataset = dataset.append(dataset_3, ignore_index=True)

In [0]:
!mkdir './drive/My Drive/636/dataset/part6/'
dataset.to_csv('./drive/My Drive/636/dataset/part6/dataset.csv')

In [0]:
dataset = dataset.drop(['frame'], axis=1)
dataset.to_csv('./drive/My Drive/636/dataset/part6/dataset_dropped.csv')

In [0]:
ls './drive/My Drive/636/data/part6_csv/'

 cleaned_data_all_pos_neg.csv   half.csv          need_norm_data.csv
 cleaned_data.csv              'half_n (1).csv'   part4.csv
 cleaned_data_half.csv          half_n.csv        pos.csv


In [0]:
CSV_FOLDER = './drive/My Drive/636/data/part6_csv/'
raw_data = pd.read_csv(CSV_FOLDER + 'half.csv')

In [0]:
raw_data.drop(raw_data.index[raw_data['vid'] == 'vid'], inplace = True)
raw_data.reset_index(drop=True)
raw_data.vid.unique()

array(['90', '89', '88', '85', '83', '56', '55', '54', '51', '49', '46'],
      dtype=object)

In [0]:
raw_data

,vid,frame,label,f_0,f_1,f_2,f_3,f_4,f_5,f_6,f_7,f_8,f_9,f_10,f_11,f_12,f_13,f_14,f_15,f_16,f_17,f_18,f_19,f_20,f_21,f_22,f_23,f_24,f_25,f_26,f_27,f_28,f_29,f_30,f_31,f_32,f_33,f_34,f_35,f_36,...,f_116,f_117,f_118,f_119,f_120,f_121,f_122,f_123,f_124,f_125,f_126,f_127,f_128,f_129,f_130,f_131,f_132,f_133,f_134,f_135,f_136,f_137,f_138,f_139,p_0,p_1,p_2,p_3,p_4,p_5,p_6,p_7,p_8,p_9,p_10,p_11,p_12,p_13,p_14,p_15
0,90,90_000000000000_keypoints.json,0,0.408179,0.271073,0.41371,0.315352,0.41832,0.361271,0.426617,0.40391,0.438602,0.443269,0.456118,0.479349,0.479166,0.502308,0.504058,0.520348,0.532638,0.518708,0.555686,0.499028,0.571359,0.469509,0.584265,0.431789,0.590719,0.39571,0.592563,0.356351,0.592563,0.316992,0.592563,0.274353,0.590719,0.234993,0.426617,0.213674,0.436758,...,0.518809,0.43015,0.50498,0.42687,0.497605,0.40391,0.517887,0.38259,0.526184,0.37931,0.53356,0.377671,0.548311,0.38915,0.534482,0.39407,0.526184,0.40063,0.518809,0.40227,0.467181,0.234993,0.550154,0.218594,0.519701,0.292111,0.5028,0.641087,0.337356,0.662834,0.0,0.0,0.0,0.0,0.674353,0.616473,0.697367,0.995329,0.0,0.0
1,90,90_000000000001_keypoints.json,0,0.407593,0.270119,0.414052,0.317718,0.418666,0.362034,0.427893,0.403068,0.438965,0.444101,0.457419,0.480211,0.479563,0.506472,0.504476,0.521244,0.53308,0.521244,0.555224,0.504831,0.57091,0.470363,0.583828,0.437536,0.590287,0.399785,0.592132,0.358752,0.592132,0.319359,0.592132,0.275043,0.590287,0.237292,0.426047,0.215955,0.437119,...,0.519239,0.432612,0.504476,0.429329,0.49894,0.409633,0.518316,0.389937,0.526621,0.388296,0.534002,0.385013,0.548765,0.389937,0.534925,0.401426,0.526621,0.403068,0.518316,0.403068,0.467568,0.240575,0.550611,0.217596,0.521136,0.292195,0.501266,0.641032,0.337368,0.662822,0.0,0.0,0.0,0.0,0.674366,0.613809,0.698881,0.995328,0.0,0.0
2,90,90_000000000002_keypoints.json,0,0.407543,0.271756,0.414022,0.321155,0.419576,0.363967,0.427907,0.405133,0.439015,0.444653,0.458454,0.480879,0.48067,0.505578,0.504738,0.522044,0.533434,0.522044,0.55565,0.502285,0.571386,0.470999,0.584345,0.434773,0.590825,0.3969,0.592676,0.357381,0.592676,0.317862,0.592676,0.275049,0.590825,0.237177,0.426056,0.219064,0.437164,...,0.519548,0.431479,0.505664,0.428186,0.500109,0.408427,0.518623,0.39196,0.526954,0.390314,0.534359,0.388667,0.54917,0.39196,0.535285,0.3969,0.526954,0.403487,0.518623,0.403487,0.467711,0.242116,0.550096,0.219064,0.519713,0.292351,0.502808,0.64105,0.337376,0.662809,0.0,0.0,0.0,0.0,0.674373,0.616452,0.698875,0.99533,0.0,0.0
3,90,90_000000000003_keypoints.json,0,0.407906,0.27391,0.414377,0.321599,0.419923,0.364353,0.428243,0.405464,0.439336,0.44493,0.457825,0.479463,0.479086,0.504129,0.504046,0.522218,0.532703,0.520573,0.555813,0.50084,0.571529,0.471241,0.585395,0.433419,0.591866,0.397242,0.59279,0.356131,0.59279,0.316665,0.59279,0.275555,0.590941,0.236089,0.42547,0.219645,0.436563,...,0.519761,0.43013,0.50497,0.425197,0.499424,0.407108,0.518836,0.392308,0.526232,0.390664,0.534552,0.38902,0.549342,0.392308,0.535476,0.393953,0.527156,0.400531,0.518836,0.402175,0.467993,0.244311,0.550267,0.219645,0.519714,0.294884,0.501268,0.641064,0.337383,0.662762,0.0,0.0,0.0,0.0,0.674379,0.616443,0.698866,0.99533,0.0,0.0
4,90,90_000000000004_keypoints.json,0,0.407973,0.274407,0.41444,0.322071,0.420908,0.364803,0.429223,0.405893,0.439387,0.445338,0.45879,0.479853,0.48004,0.502863,0.504987,0.520942,0.533629,0.519298,0.555803,0.499576,0.57151,0.469992,0.58537,0.433833,0.592761,0.396031,0.593685,0.354942,0.593685,0.315497,0.592761,0.274407,0.590913,0.234962,0.426452,0.22017,0.437539,...,0.51977,0.430546,0.504987,0.427259,0.499443,0.407536,0.518846,0.392744,0.527161,0.3911,0.534553,0.387813,0.549336,0.3911,0.535477,0.394388,0.527161,0.400962,0.518846,0.402605,0.468029,0.244823,0.551184,0.22017,0.519707,0.294912,0.501276,0.64106,0.337397,0.660108,0.0,0.0,0.0,0.0,0.674366,0.613797,0.698876,0.99533,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

In [0]:
raw_data['label'] = raw_data['label'].astype(int)
raw_data['vid'] = raw_data['vid'].astype(int)
cols = list(raw_data.columns)
cols.remove('vid')
cols.remove('frame')
cols.remove('label')
for col in cols:
    raw_data[col] = raw_data[col].astype(float)

In [0]:
raw_dataset = pd.read_csv('./drive/My Drive/636/dataset/part6/dataset.csv', index_col=0)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
raw_dataset = raw_dataset.append(raw_data, ignore_index=True)

In [0]:
raw_dataset = raw_dataset.sort_values(by=['vid'], ignore_index=True)

In [0]:
raw_dataset.to_csv('./drive/My Drive/636/dataset/part6/dataset_all.csv')

In [0]:
dataset = raw_dataset.drop(['frame'], axis=1)
dataset.to_csv('./drive/My Drive/636/dataset/part6/dataset_all_dropped.csv')

divide to train and test

In [0]:
import json
import pandas as pd
import numpy as np
import os, os.path
import csv
dataset = pd.read_csv('./drive/My Drive/636/dataset/part6/dataset_all_dropped.csv', index_col=0)

In [0]:
dataset.index[dataset['label'] == 1].shape[0]

67430

In [0]:
dataset.index[dataset['label'] == 0].shape[0]

82303

In [0]:
dataset.shape[0] * 0.15

22459.95

In [0]:
for i in dataset.vid[dataset['label'] == 0].drop_duplicates():
  print(i)

0
1
2
5
6
8
9
10
11
12
13
14
16
19
22
23
24
25
26
36
37
39
42
45
46
48
49
50
51
52
53
54
55
56
59
60
61
62
63
65
66
67
69
83
85
88
89
90
91
92
93
95
96
98
100
101
102
104
106
107
112
113
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298
299
300
301
302
303
304
305
306
307
308
309
310
311
312
313
314
315
316
317
318
319


In [0]:
dataset.vid.unique()

array([  0,   1,   2,   5,   6,   8,   9,  10,  11,  12,  13,  14,  16,
        18,  19,  22,  23,  24,  25,  26,  28,  29,  30,  31,  32,  33,
        34,  35,  36,  37,  38,  39,  40,  41,  42,  43,  45,  46,  47,
        48,  49,  50,  51,  52,  53,  54,  55,  56,  57,  58,  59,  60,
        61,  62,  63,  64,  65,  66,  67,  68,  69,  83,  84,  85,  86,
        87,  88,  89,  90,  91,  92,  93,  95,  96,  97,  98,  99, 100,
       101, 102, 103, 104, 105, 106, 107, 109, 111, 112, 113, 155, 156,
       157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169,
       170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182,
       183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195,
       196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208,
       209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221,
       222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234,
       235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 24

In [0]:
for i in dataset.vid.unique():
  print(i, dataset.index[dataset['vid'] == i].shape[0])

In [0]:
part3 = [i for i in range(155, 320)]

In [0]:
part2 = [i for i in range(18, 114)]

In [0]:
part1 = [i for i in range(1, 18)]

In [0]:
dataset1 = dataset.loc[dataset['vid'].isin(part1)]
dataset2 = dataset.loc[dataset['vid'].isin(part2)]
dataset3 = dataset.loc[dataset['vid'].isin(part3)]

In [0]:
dataset1.shape[0] * 0.2
# 0 1

18768.2

In [0]:
dataset2.shape[0] * 0.2
# 18 - 43

9356.0

In [0]:
dataset.iloc[dataset1.shape[0] + 9356]

vid      43.000000
label     1.000000
f_0       0.391844
f_1       0.353910
f_2       0.390769
           ...    
p_11      0.898590
p_12      0.000000
p_13      0.000000
p_14      0.000000
p_15      0.000000
Name: 103197, Length: 158, dtype: float64

In [0]:
dataset2.iloc[9356]

vid      46.000000
label     1.000000
f_0       0.404667
f_1       0.375625
f_2       0.403560
           ...    
p_11      0.858989
p_12      0.000000
p_13      0.000000
p_14      0.000000
p_15      0.000000
Name: 105528, Length: 158, dtype: float64

In [0]:
dataset3.shape[0] * 0.2

1356.2

In [0]:
dataset3.iloc[1356]

vid      190.000000
label      1.000000
f_0        0.376445
f_1        0.431013
f_2        0.376445
            ...    
p_11       0.886199
p_12       0.000000
p_13       0.000000
p_14       0.000000
p_15       0.000000
Name: 144308, Length: 158, dtype: float64

In [0]:
test_list_2 = [i for i in range(299, 320)]
test_list_3 = [i for i in range(219, 240)]
test_list = test_list_2 + test_list_3 + [0, 1, 28, 29, 31, 34, 44, 62, 63, 100, 101, 102, 111, 112, 113]

In [0]:
testset = dataset.loc[dataset['vid'].isin(test_list)]

In [0]:
testset = testset.reset_index(drop=True)

In [0]:
testset = testset[(testset.T != 0).any()]

In [0]:
testset = testset.reset_index(drop=True)

In [0]:
train_list = [i for i in range(320) if (i not in test_list)]
trainset = dataset.loc[dataset['vid'].isin(train_list)]
trainset = trainset[(trainset.T != 0).any()]
trainset = trainset.reset_index(drop=True)

In [0]:
drop_list = [i for i in range(100)]
trainset.drop(trainset.index[drop_list], inplace=True)

In [0]:
trainset = trainset.reset_index(drop=True)

In [0]:
!rm './drive/My Drive/636/dataset/part6/testset.csv'
!rm './drive/My Drive/636/dataset/part6/trainset.csv'

testset.to_csv('./drive/My Drive/636/dataset/part6/testset.csv')
trainset.to_csv('./drive/My Drive/636/dataset/part6/trainset.csv')

In [0]:
# trainset.index[trainset['label'] == 1].shape[0]

In [0]:
trainset.shape

(127308, 158)

In [0]:
testset.index[testset['label'] == 1].shape[0]

9058